In [1]:
import os
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_auc_score, average_precision_score, classification_report, accuracy_score

import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
from torchmetrics.regression import MeanSquaredError, MeanAbsoluteError
from torchmetrics.classification import BinaryAUROC, BinaryAveragePrecision
from torchmetrics.classification import BinaryAccuracy, BinaryPrecision, BinaryRecall, BinaryF1Score

import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
from lightning.pytorch.callbacks import Callback

import optuna
from optuna.integration import PyTorchLightningPruningCallback
from optuna.trial import FixedTrial

c:\Users\mulso\anaconda3\envs\arcr\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 데이터셋 정의

In [2]:
DATA_MAKER = "jiseock"
if DATA_MAKER == "jiseock":
    DATA_PATH = "../dataset/jiseock"
else:
    DATA_PATH = "../dataset/yeonseo"

X_train = pd.read_csv(f"{DATA_PATH}/X_train.csv")
y_train = pd.read_csv(f"{DATA_PATH}/y_train.csv")

In [3]:
X_train

,나이,"성별 (M:1,F:2)","Rt:1,Lt:2",Height,Weight,"Tearsize (AP,cm)",Tearsize (ML),Tearsize (retraction),"흡연여부 (비흡연:1,흡연:2)","흡연여부 (비흡연:1,흡연:2) Missing flag",...,6M Goutallier (ISP),6M Goutallier (TM),Pre Goutallier (SSP) Missing flag,Pre Goutallier (SSC) Missing flag,Pre Goutallier (ISP) Missing flag,Pre Goutallier (TM) Missing flag,6M Goutallier (SSP) Missing flag,6M Goutallier (SSC) Missing flag,6M Goutallier (ISP) Missing flag,6M Goutallier (TM) Missing flag
0,70,2,1,-0.112260,-0.239010,0.529347,1.299314,1.236586,1.0,1.0,...,-0.229042,-0.201304,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,61,1,1,0.429514,-0.072467,-0.673177,-0.693529,-0.671659,1.0,1.0,...,-0.229042,-0.201304,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,73,2,1,-0.344449,-0.488825,-0.913682,-0.927981,-0.896158,1.0,0.0,...,-0.229042,-0.201304,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,60,1,2,0.584307,0.268947,-0.071915,-0.693529,-0.671659,1.0,1.0,...,-0.229042,-0.201304,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,75,2,2,-0.050343,-0.209865,0.529347,0.478732,0.450838,1.0,1.0,...,-0.229042,-0.201304,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7173,72,2,2,-0.371571,-1.414487,0.014274,-0.777214,-0.751792,1.0,1.0,...,-0.229042,-0.201304,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
7174,80,1,2,0.038328,0.032052,0.070864,-0.498668,-0.485071,1.0,1.0,...,-0.229042,-0.201304,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7175,83,2,1,0.778539,0.094077,-0.174047,-0.206960,-0.205745,1.0,1.0,...,-0.229042,-0.201304,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7176,65,2,1,0.048063,-0.588316,-1.114365,-1.123613,-1.083486,1.0,1.0,...,-0.229042,-0.201304,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [4]:
y_train

,POD 6M retear
0,0
1,0
2,0
3,0
4,0
...,...
7173,1
7174,1
7175,1
7176,1


In [5]:
columns = list(X_train.columns)
columns

['나이',
 '성별 (M:1,F:2)',
 'Rt:1,Lt:2',
 'Height',
 'Weight',
 'Tearsize (AP,cm)',
 'Tearsize (ML)',
 'Tearsize (retraction)',
 '흡연여부 (비흡연:1,흡연:2)',
 '흡연여부 (비흡연:1,흡연:2) Missing flag',
 'Hospital 0',
 'Hospital 1',
 'Hospital 2',
 'Hospital 3',
 'Hospital 4',
 'Hospital 5',
 'Hospital 6',
 'Disease 0',
 'Disease 1',
 'Disease 2',
 'Disease 3',
 'Disease 4',
 'Disease 5',
 'Disease 6',
 'Disease 7',
 '0M ASES',
 '0M CSS',
 '0M ERabd',
 '0M ERside',
 '0M FF',
 '0M IR',
 '0M KSS',
 '0M MMTgrade',
 '0M MMTsec',
 '0M VAS(activity)',
 '0M VAS(resting)',
 '0M add',
 '2M ERabd',
 '2M ERside',
 '2M FF',
 '2M IR',
 '2M MMTgrade',
 '2M MMTsec',
 '2M add',
 '3M ASES',
 '3M CSS',
 '3M ERabd',
 '3M ERside',
 '3M FF',
 '3M IR',
 '3M KSS',
 '3M MMTgrade',
 '3M MMTsec',
 '3M VAS(activity)',
 '3M VAS(resting)',
 '3M add',
 '4M ASES',
 '4M CSS',
 '4M ERabd',
 '4M ERside',
 '4M FF',
 '4M IR',
 '4M KSS',
 '4M MMTgrade',
 '4M MMTsec',
 '4M VAS(activity)',
 '4M VAS(resting)',
 '4M add',
 '6M ASES',
 '6M CSS',
 

In [6]:
static_columns = columns[:25]

# static 데이터 칼럼
static_columns

['나이',
 '성별 (M:1,F:2)',
 'Rt:1,Lt:2',
 'Height',
 'Weight',
 'Tearsize (AP,cm)',
 'Tearsize (ML)',
 'Tearsize (retraction)',
 '흡연여부 (비흡연:1,흡연:2)',
 '흡연여부 (비흡연:1,흡연:2) Missing flag',
 'Hospital 0',
 'Hospital 1',
 'Hospital 2',
 'Hospital 3',
 'Hospital 4',
 'Hospital 5',
 'Hospital 6',
 'Disease 0',
 'Disease 1',
 'Disease 2',
 'Disease 3',
 'Disease 4',
 'Disease 5',
 'Disease 6',
 'Disease 7']

In [7]:
seq_columns = columns[25:-16]

# 시퀀셜 데이터 관련 칼럼들
seq_columns_0M = seq_columns[:12]
seq_columns_2M = seq_columns[12:19]
seq_columns_3M = seq_columns[19:31]
seq_columns_4M = seq_columns[31:43]
seq_columns_6M = seq_columns[43:]

seq_columns_all = [seq_columns_0M, seq_columns_2M, seq_columns_3M, seq_columns_4M, seq_columns_6M]

for seq_col in seq_columns_all:
    print(seq_col)

['0M ASES', '0M CSS', '0M ERabd', '0M ERside', '0M FF', '0M IR', '0M KSS', '0M MMTgrade', '0M MMTsec', '0M VAS(activity)', '0M VAS(resting)', '0M add']
['2M ERabd', '2M ERside', '2M FF', '2M IR', '2M MMTgrade', '2M MMTsec', '2M add']
['3M ASES', '3M CSS', '3M ERabd', '3M ERside', '3M FF', '3M IR', '3M KSS', '3M MMTgrade', '3M MMTsec', '3M VAS(activity)', '3M VAS(resting)', '3M add']
['4M ASES', '4M CSS', '4M ERabd', '4M ERside', '4M FF', '4M IR', '4M KSS', '4M MMTgrade', '4M MMTsec', '4M VAS(activity)', '4M VAS(resting)', '4M add']
['6M ASES', '6M CSS', '6M ERabd', '6M ERside', '6M FF', '6M IR', '6M KSS', '6M MMTgrade', '6M MMTsec', '6M VAS(activity)', '6M VAS(resting)', '6M add']


In [8]:
goutallier_columns = columns[-16:]

# goutaliar 관련 칼럼들
goutallier_columns_0M = goutallier_columns [:4]
goutallier_columns_6M = goutallier_columns [4:8]
goutallier_columns_0M_missing = goutallier_columns [8:12]
goutallier_columns_6M_missing = goutallier_columns [12:]

print(goutallier_columns_0M)
print(goutallier_columns_6M)
print(goutallier_columns_0M_missing)
print(goutallier_columns_6M_missing)

['Pre Goutallier (SSP)', 'Pre Goutallier (SSC)', 'Pre Goutallier (ISP)', 'Pre Goutallier (TM)']
['6M Goutallier (SSP)', '6M Goutallier (SSC)', '6M Goutallier (ISP)', '6M Goutallier (TM)']
['Pre Goutallier (SSP) Missing flag', 'Pre Goutallier (SSC) Missing flag', 'Pre Goutallier (ISP) Missing flag', 'Pre Goutallier (TM) Missing flag']
['6M Goutallier (SSP) Missing flag', '6M Goutallier (SSC) Missing flag', '6M Goutallier (ISP) Missing flag', '6M Goutallier (TM) Missing flag']


In [9]:
len(columns) == len(static_columns) + len(seq_columns) + len(goutallier_columns)

True

In [10]:
label_column = "POD 6M retear"
output_columns = ["6M ASES", "6M CSS", "6M KSS", "6M VAS(activity)", "6M VAS(resting)"]
input_columns = static_columns + [column for column in seq_columns if column not in output_columns] + goutallier_columns

positive_rate = y_train[label_column].mean()
if positive_rate <= 0 or positive_rate >= 1:
    cls_pos_weight = 1.0
else:
    cls_pos_weight = float((1 - positive_rate) / positive_rate)

In [11]:
output_columns

['6M ASES', '6M CSS', '6M KSS', '6M VAS(activity)', '6M VAS(resting)']

In [12]:
input_columns

['나이',
 '성별 (M:1,F:2)',
 'Rt:1,Lt:2',
 'Height',
 'Weight',
 'Tearsize (AP,cm)',
 'Tearsize (ML)',
 'Tearsize (retraction)',
 '흡연여부 (비흡연:1,흡연:2)',
 '흡연여부 (비흡연:1,흡연:2) Missing flag',
 'Hospital 0',
 'Hospital 1',
 'Hospital 2',
 'Hospital 3',
 'Hospital 4',
 'Hospital 5',
 'Hospital 6',
 'Disease 0',
 'Disease 1',
 'Disease 2',
 'Disease 3',
 'Disease 4',
 'Disease 5',
 'Disease 6',
 'Disease 7',
 '0M ASES',
 '0M CSS',
 '0M ERabd',
 '0M ERside',
 '0M FF',
 '0M IR',
 '0M KSS',
 '0M MMTgrade',
 '0M MMTsec',
 '0M VAS(activity)',
 '0M VAS(resting)',
 '0M add',
 '2M ERabd',
 '2M ERside',
 '2M FF',
 '2M IR',
 '2M MMTgrade',
 '2M MMTsec',
 '2M add',
 '3M ASES',
 '3M CSS',
 '3M ERabd',
 '3M ERside',
 '3M FF',
 '3M IR',
 '3M KSS',
 '3M MMTgrade',
 '3M MMTsec',
 '3M VAS(activity)',
 '3M VAS(resting)',
 '3M add',
 '4M ASES',
 '4M CSS',
 '4M ERabd',
 '4M ERside',
 '4M FF',
 '4M IR',
 '4M KSS',
 '4M MMTgrade',
 '4M MMTsec',
 '4M VAS(activity)',
 '4M VAS(resting)',
 '4M add',
 '6M ERabd',
 '6M ERside

In [13]:
X_train[input_columns]

,나이,"성별 (M:1,F:2)","Rt:1,Lt:2",Height,Weight,"Tearsize (AP,cm)",Tearsize (ML),Tearsize (retraction),"흡연여부 (비흡연:1,흡연:2)","흡연여부 (비흡연:1,흡연:2) Missing flag",...,6M Goutallier (ISP),6M Goutallier (TM),Pre Goutallier (SSP) Missing flag,Pre Goutallier (SSC) Missing flag,Pre Goutallier (ISP) Missing flag,Pre Goutallier (TM) Missing flag,6M Goutallier (SSP) Missing flag,6M Goutallier (SSC) Missing flag,6M Goutallier (ISP) Missing flag,6M Goutallier (TM) Missing flag
0,70,2,1,-0.112260,-0.239010,0.529347,1.299314,1.236586,1.0,1.0,...,-0.229042,-0.201304,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,61,1,1,0.429514,-0.072467,-0.673177,-0.693529,-0.671659,1.0,1.0,...,-0.229042,-0.201304,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,73,2,1,-0.344449,-0.488825,-0.913682,-0.927981,-0.896158,1.0,0.0,...,-0.229042,-0.201304,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,60,1,2,0.584307,0.268947,-0.071915,-0.693529,-0.671659,1.0,1.0,...,-0.229042,-0.201304,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,75,2,2,-0.050343,-0.209865,0.529347,0.478732,0.450838,1.0,1.0,...,-0.229042,-0.201304,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7173,72,2,2,-0.371571,-1.414487,0.014274,-0.777214,-0.751792,1.0,1.0,...,-0.229042,-0.201304,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
7174,80,1,2,0.038328,0.032052,0.070864,-0.498668,-0.485071,1.0,1.0,...,-0.229042,-0.201304,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7175,83,2,1,0.778539,0.094077,-0.174047,-0.206960,-0.205745,1.0,1.0,...,-0.229042,-0.201304,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7176,65,2,1,0.048063,-0.588316,-1.114365,-1.123613,-1.083486,1.0,1.0,...,-0.229042,-0.201304,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [14]:
pd.concat([y_train, X_train[output_columns]], axis=1)

,POD 6M retear,6M ASES,6M CSS,6M KSS,6M VAS(activity),6M VAS(resting)
0,0,1.377047,1.448052,0.201114,1.201822,1.202527
1,0,0.756417,0.258163,1.336235,-1.788307,-1.761167
2,0,-0.655014,1.087954,-1.004221,1.201822,1.202527
3,0,-0.594953,-0.054966,-0.699962,1.201822,1.202527
4,0,0.566225,-1.526670,-0.126550,0.205112,0.214629
...,...,...,...,...,...,...
7173,1,0.576435,-0.284644,0.671297,0.560877,0.567248
7174,1,-0.788472,0.328121,-0.568735,0.205112,0.214629
7175,1,0.678394,-0.617836,-0.355947,2.198532,2.190425
7176,1,0.932377,0.515679,0.898310,0.205112,0.214629


In [15]:
def get_dataset(split):
  assert split in ["train", "val", "test"]
  
  X_file_name = f"{DATA_PATH}/X_{split}.csv"
  y_file_name = f"{DATA_PATH}/y_{split}.csv"

  X = pd.read_csv(X_file_name)
  y = pd.read_csv(y_file_name)
  
  # static 데이터
  X_static_tensor = torch.tensor(X[static_columns].to_numpy(), dtype=torch.float32)

  # 시기별 sequential 데이터
  X_seq_tensor_0M = torch.tensor(X[seq_columns_0M].to_numpy(), dtype=torch.float32)
  X_seq_tensor_2M = torch.tensor(X[seq_columns_2M].to_numpy(), dtype=torch.float32)
  X_seq_tensor_3M = torch.tensor(X[seq_columns_3M].to_numpy(), dtype=torch.float32)
  X_seq_tensor_4M = torch.tensor(X[seq_columns_4M].to_numpy(), dtype=torch.float32)
  X_seq_tensor_6M = torch.tensor(X[seq_columns_6M].to_numpy(), dtype=torch.float32)
  
  #0M, 6M goutalier 데이터
  X_goutalier_tensor_0M = torch.tensor(X[goutallier_columns_0M + goutallier_columns_0M_missing].to_numpy(), dtype=torch.float32)
  X_goutalier_tensor_6M = torch.tensor(X[goutallier_columns_6M + goutallier_columns_6M_missing].to_numpy(), dtype=torch.float32)
  
  # 전체 인풋 데이터
  X_tensor = torch.tensor(X[input_columns].to_numpy(), dtype=torch.float32)
  
  # 6M 예측 데이터
  y_tensor = torch.tensor(pd.concat([y, X[output_columns]], axis=1).to_numpy(), dtype=torch.float32)

  return TensorDataset(X_tensor, X_static_tensor, X_seq_tensor_0M, X_seq_tensor_2M, X_seq_tensor_3M, X_seq_tensor_4M, X_seq_tensor_6M, X_goutalier_tensor_0M, X_goutalier_tensor_6M, y_tensor)

In [16]:
trainset = get_dataset("train")
valset = get_dataset("val")
testset = get_dataset("test")

print("기존 데이터셋 구조 확인")
print(f"Trainset size: {len(trainset)}")
print(f"Validset size: {len(valset)}") # jiseock
print(f"Testset size: {len(testset)}")

기존 데이터셋 구조 확인
Trainset size: 7178
Validset size: 647
Testset size: 100


In [17]:
# 특징 크기 확인
static_features = len(static_columns)
seq_features_0M = len(seq_columns_0M)
seq_features_2M = len(seq_columns_2M)
seq_features_3M = len(seq_columns_3M)
seq_features_4M = len(seq_columns_4M)
seq_features_6M = len(seq_columns_6M)
goutallier_features_0M = len(goutallier_columns_0M) + len(goutallier_columns_0M_missing)
goutallier_features_6M = len(goutallier_columns_6M) + len(goutallier_columns_6M_missing)

print(f"Static features: {static_features}")
print(f"0M features: {seq_features_0M}, 2M features: {seq_features_2M}, 3M features: {seq_features_3M}")
print(f"4M features: {seq_features_4M}, 6M features: {seq_features_6M}")
print(f"0M Goutallier features: {goutallier_features_0M}, 6M Goutallier features: {goutallier_features_6M}")

Static features: 25
0M features: 12, 2M features: 7, 3M features: 12
4M features: 12, 6M features: 12
0M Goutallier features: 8, 6M Goutallier features: 8


In [18]:
# 각 모델별 데이터셋 생성 함수
def get_dataset_model1(split):
    """Model 1: static + 0M + 0M_goutallier → 2M"""
    assert split in ["train", "val", "test"]
    
    X_file_name = f"{DATA_PATH}/X_{split}.csv"
    y_file_name = f"{DATA_PATH}/y_{split}.csv"
    
    X = pd.read_csv(X_file_name)
    y = pd.read_csv(y_file_name)

    # 입력: static + 0M + 0M_goutallier
    X_static = torch.tensor(X[static_columns].to_numpy(), dtype=torch.float32)
    X_0M = torch.tensor(X[seq_columns_0M].to_numpy(), dtype=torch.float32)
    X_0M_goutallier = torch.tensor(X[goutallier_columns_0M + goutallier_columns_0M_missing].to_numpy(), dtype=torch.float32)
    
    # 출력: 2M
    y_2M = torch.tensor(X[seq_columns_2M].to_numpy(), dtype=torch.float32)
    
    return TensorDataset(X_static, X_0M, X_0M_goutallier, y_2M)

def get_dataset_model2(split):
    """Model 2: static + 0M + 2M + 0M_goutallier → 3M"""
    assert split in ["train", "val", "test"]
    
    X_file_name = f"{DATA_PATH}/X_{split}.csv"
    y_file_name = f"{DATA_PATH}/y_{split}.csv"
    
    X = pd.read_csv(X_file_name)
    y = pd.read_csv(y_file_name)

    # 입력: static + 0M + 2M + 0M_goutallier
    X_static = torch.tensor(X[static_columns].to_numpy(), dtype=torch.float32)
    X_0M = torch.tensor(X[seq_columns_0M].to_numpy(), dtype=torch.float32)
    X_2M = torch.tensor(X[seq_columns_2M].to_numpy(), dtype=torch.float32)
    X_0M_goutallier = torch.tensor(X[goutallier_columns_0M + goutallier_columns_0M_missing].to_numpy(), dtype=torch.float32)
    
    # 출력: 3M
    y_3M = torch.tensor(X[seq_columns_3M].to_numpy(), dtype=torch.float32)
    
    return TensorDataset(X_static, X_0M, X_2M, X_0M_goutallier, y_3M)

def get_dataset_model3(split):
    """Model 3: static + 0M + 2M + 3M + 0M_goutallier → 4M"""
    assert split in ["train", "val", "test"]
    
    X_file_name = f"{DATA_PATH}/X_{split}.csv"
    y_file_name = f"{DATA_PATH}/y_{split}.csv"
    
    X = pd.read_csv(X_file_name)
    y = pd.read_csv(y_file_name)

    # 입력: static + 0M + 2M + 3M + 0M_goutallier
    X_static = torch.tensor(X[static_columns].to_numpy(), dtype=torch.float32)
    X_0M = torch.tensor(X[seq_columns_0M].to_numpy(), dtype=torch.float32)
    X_2M = torch.tensor(X[seq_columns_2M].to_numpy(), dtype=torch.float32)
    X_3M = torch.tensor(X[seq_columns_3M].to_numpy(), dtype=torch.float32)
    X_0M_goutallier = torch.tensor(X[goutallier_columns_0M + goutallier_columns_0M_missing].to_numpy(), dtype=torch.float32)
    
    # 출력: 4M
    y_4M = torch.tensor(X[seq_columns_4M].to_numpy(), dtype=torch.float32)
    
    return TensorDataset(X_static, X_0M, X_2M, X_3M, X_0M_goutallier, y_4M)

def get_dataset_model4(split):
    """Model 4: static + 0M + 2M + 3M + 4M + 0M_goutallier → 6M + y + 6M_goutallier"""
    assert split in ["train", "val", "test"]
    
    X_file_name = f"{DATA_PATH}/X_{split}.csv"
    y_file_name = f"{DATA_PATH}/y_{split}.csv"
    
    X = pd.read_csv(X_file_name)
    y = pd.read_csv(y_file_name)

    # 입력: static + 0M + 2M + 3M + 4M + 0M_goutallier
    X_static = torch.tensor(X[static_columns].to_numpy(), dtype=torch.float32)
    X_0M = torch.tensor(X[seq_columns_0M].to_numpy(), dtype=torch.float32)
    X_2M = torch.tensor(X[seq_columns_2M].to_numpy(), dtype=torch.float32)
    X_3M = torch.tensor(X[seq_columns_3M].to_numpy(), dtype=torch.float32)
    X_4M = torch.tensor(X[seq_columns_4M].to_numpy(), dtype=torch.float32)
    X_0M_goutallier = torch.tensor(X[goutallier_columns_0M + goutallier_columns_0M_missing].to_numpy(), dtype=torch.float32)
    
    # 출력: 6M + y + 6M_goutallier
    y_6M = torch.tensor(X[seq_columns_6M].to_numpy(), dtype=torch.float32)
    y_label = torch.tensor(y[label_column].to_numpy(), dtype=torch.float32).unsqueeze(1)
    y_6M_goutallier = torch.tensor(X[goutallier_columns_6M + goutallier_columns_6M_missing].to_numpy(), dtype=torch.float32)
    
    # 결합: [6M features (12) + y (1) + 6M_goutallier (8)] = 21
    y_combined = torch.cat([y_6M, y_label, y_6M_goutallier], dim=1)
    
    return TensorDataset(X_static, X_0M, X_2M, X_3M, X_4M, X_0M_goutallier, y_combined)

# 데이터셋 생성
trainset_model1 = get_dataset_model1("train")
testset_model1 = get_dataset_model1("test")
valset_model1 = get_dataset_model1("val")

trainset_model2 = get_dataset_model2("train")
testset_model2 = get_dataset_model2("test")
valset_model2 = get_dataset_model2("val")

trainset_model3 = get_dataset_model3("train")
testset_model3 = get_dataset_model3("test")
valset_model3 = get_dataset_model3("val")

trainset_model4 = get_dataset_model4("train")
testset_model4 = get_dataset_model4("test")
valset_model4 = get_dataset_model4("val")

print(f"Model 1 - Train: {len(trainset_model1)}, Test: {len(testset_model1)}, Valid: {len(valset_model1)}")
print(f"Model 2 - Train: {len(trainset_model2)}, Test: {len(testset_model2)}, Valid: {len(valset_model2)}")
print(f"Model 3 - Train: {len(trainset_model3)}, Test: {len(testset_model3)}, Valid: {len(valset_model3)}")
print(f"Model 4 - Train: {len(trainset_model4)}, Test: {len(testset_model4)}, Valid: {len(valset_model4)}")


Model 1 - Train: 7178, Test: 100, Valid: 647
Model 2 - Train: 7178, Test: 100, Valid: 647
Model 3 - Train: 7178, Test: 100, Valid: 647
Model 4 - Train: 7178, Test: 100, Valid: 647


In [19]:
class LossHistoryCallback(Callback):
    def __init__(self):
        super().__init__()
        self.train_losses = []
        self.test_losses = []
    
    def on_train_epoch_end(self, trainer, pl_module):
        if len(self.train_losses) == 0:
            print(f"[Train] Available metrics: {list(trainer.callback_metrics.keys())}")
        
        train_loss = trainer.callback_metrics.get('train/loss_epoch')
        if train_loss is not None:
            self.train_losses.append(train_loss.item())
        else:
            print(f"Warning: train/loss_epoch not found!")
    
    def on_validation_epoch_end(self, trainer, pl_module):
        if len(self.test_losses) == 0:
            print(f"[Val] Available metrics: {list(trainer.callback_metrics.keys())}")
        
        val_loss = trainer.callback_metrics.get('val/loss')
        if val_loss is not None:
            self.test_losses.append(val_loss.item())
        else:
            print(f"Warning: val/loss not found!")

class CleanupCheckpointCallback(Callback):
    """학습 종료 후 최고 성능 체크포인트만 남기고 나머지 제거"""
    def __init__(self, checkpoint_dir, filename_prefix):
        super().__init__()
        self.checkpoint_dir = checkpoint_dir
        self.filename_prefix = filename_prefix
    
    def on_train_end(self, trainer, pl_module):
        import os
        import glob
        
        # 체크포인트 디렉토리의 모든 파일 확인 (filename_prefix로 시작하는 파일만)
        checkpoint_files = glob.glob(os.path.join(self.checkpoint_dir, f"{self.filename_prefix}*.ckpt"))
        
        if len(checkpoint_files) == 0:
            return
        
        # 최고 성능 체크포인트 찾기 (ModelCheckpoint가 저장한 best 체크포인트)
        best_checkpoint = None
        for callback in trainer.callbacks:
            if isinstance(callback, ModelCheckpoint):
                best_checkpoint = callback.best_model_path
                break
        
        # best_checkpoint가 filename_prefix로 시작하는지 확인
        # 그리고 checkpoint_files에 포함되어 있는지 확인
        if best_checkpoint and os.path.exists(best_checkpoint):
            best_checkpoint_basename = os.path.basename(best_checkpoint)
            # filename_prefix로 시작하는지 확인
            if not best_checkpoint_basename.startswith(self.filename_prefix):
                # 다른 모델의 체크포인트인 경우, checkpoint_files에서만 정리
                best_checkpoint = None
            # checkpoint_files에 포함되어 있는지 확인
            elif best_checkpoint not in checkpoint_files:
                # 경로가 다를 수 있으므로 정규화하여 비교
                best_checkpoint_normalized = os.path.normpath(best_checkpoint)
                checkpoint_files_normalized = [os.path.normpath(f) for f in checkpoint_files]
                if best_checkpoint_normalized not in checkpoint_files_normalized:
                    best_checkpoint = None
        
        # 최고 성능 체크포인트가 있으면 나머지 제거
        if best_checkpoint and os.path.exists(best_checkpoint):
            removed_count = 0
            for checkpoint_file in checkpoint_files:
                # 정규화된 경로로 비교
                if os.path.normpath(checkpoint_file) != os.path.normpath(best_checkpoint):
                    try:
                        os.remove(checkpoint_file)
                        removed_count += 1
                    except Exception as e:
                        print(f"Warning: 체크포인트 삭제 실패 {checkpoint_file}: {e}")
            
            if removed_count > 0:
                print(f"[체크포인트 정리] {removed_count}개의 체크포인트 제거 완료. 최고 성능 체크포인트만 유지: {os.path.basename(best_checkpoint)}")
        else:
            # best 체크포인트를 찾지 못한 경우, 파일명으로 최신 것만 남기기
            if checkpoint_files:
                # 파일 수정 시간 기준으로 정렬
                checkpoint_files.sort(key=lambda x: os.path.getmtime(x), reverse=True)
                best_checkpoint = checkpoint_files[0]
                
                removed_count = 0
                for checkpoint_file in checkpoint_files[1:]:
                    try:
                        os.remove(checkpoint_file)
                        removed_count += 1
                    except Exception as e:
                        print(f"Warning: 체크포인트 삭제 실패 {checkpoint_file}: {e}")
                
                if removed_count > 0:
                    print(f"[체크포인트 정리] {removed_count}개의 체크포인트 제거 완료. 최신 체크포인트만 유지: {os.path.basename(best_checkpoint)}")


### Optuna 모델 최적화

In [20]:
def create_encoder(trial, input_dim, name_prefix, min_units=32, max_units=256, min_layers=1, max_layers=10):
    """동적으로 인코더 생성 - NAS 기반"""
    n_layers = trial.suggest_int(f'{name_prefix}_n_layers', min_layers, max_layers)
    layers = []
    prev_dim = input_dim
    
    for i in range(n_layers):
        units = trial.suggest_int(f'{name_prefix}_units_{i}', min_units, max_units, step=32)
        dropout = trial.suggest_float(f'{name_prefix}_dropout_{i}', 0.0, 0.5, step=0.05)
        use_batch_norm = trial.suggest_categorical(f'{name_prefix}_batch_norm_{i}', [True, False])
        
        layers.append(nn.Linear(prev_dim, units))
        if use_batch_norm:
            layers.append(nn.BatchNorm1d(units))
        else:
            layers.append(nn.LayerNorm(units))
        layers.append(nn.LeakyReLU())
        if dropout > 0:
            layers.append(nn.Dropout(dropout))
        
        prev_dim = units
    
    return nn.Sequential(*layers), prev_dim

def create_output_head(trial, input_dim, output_dim, name_prefix, min_units=64, max_units=512, min_layers=1, max_layers=4):
    """동적으로 출력 헤드 생성 - NAS 기반"""
    n_layers = trial.suggest_int(f'{name_prefix}_n_layers', min_layers, max_layers)
    layers = []
    prev_dim = input_dim
    
    for i in range(n_layers - 1):
        units = trial.suggest_int(f'{name_prefix}_units_{i}', min_units, max_units, step=32)
        dropout = trial.suggest_float(f'{name_prefix}_dropout_{i}', 0.0, 0.5, step=0.05)
        use_batch_norm = trial.suggest_categorical(f'{name_prefix}_batch_norm_{i}', [True, False])
        
        layers.append(nn.Linear(prev_dim, units))
        if use_batch_norm:
            layers.append(nn.BatchNorm1d(units))
        else:
            layers.append(nn.LayerNorm(units))
        layers.append(nn.LeakyReLU())
        if dropout > 0:
            layers.append(nn.Dropout(dropout))
        
        prev_dim = units
    
    # 마지막 레이어 (출력)
    layers.append(nn.Linear(prev_dim, output_dim))
    
    return nn.Sequential(*layers)

print("NAS 기반 인코더/헤드 생성 함수 정의 완료")


NAS 기반 인코더/헤드 생성 함수 정의 완료


In [21]:
# Model 1 최적화 버전
class OptimizedSequentialMLP1(L.LightningModule):
    def __init__(self, trial, static_features, seq_0M_features, goutallier_0M_features, out_features_2M):
        super().__init__()
        
        # 인코더 생성
        self.static_encoder, static_out = create_encoder(
            trial, static_features, 'static_encoder', min_units=64, max_units=256
        )
        
        self.seq_0M_encoder, seq_out = create_encoder(
            trial, seq_0M_features, 'seq_0M_encoder', min_units=64, max_units=256
        )
        
        self.goutallier_0M_encoder, goutalier_out = create_encoder(
            trial, goutallier_0M_features, 'goutallier_0M_encoder', min_units=32, max_units=128
        )
        
        # 특징 결합 후 출력 헤드
        feat_dim = static_out + seq_out + goutalier_out
        self.output_head = create_output_head(
            trial, feat_dim, out_features_2M, 'output_head', min_units=128, max_units=512
        )
        
        # 학습 파라미터
        self.lr = trial.suggest_float('lr', 1e-7, 1e-4, log=True)
        self.weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-3, log=True)
        
        self.train_mse = MeanSquaredError()
        self.val_mse = MeanSquaredError()
        self.test_mse = MeanSquaredError()
        
    def forward(self, x_static, x_0M, x_0M_goutallier):
        static_feat = self.static_encoder(x_static)
        seq_0M_feat = self.seq_0M_encoder(x_0M)
        goutallier_0M_feat = self.goutallier_0M_encoder(x_0M_goutallier)
        
        combined = torch.cat([static_feat, seq_0M_feat, goutallier_0M_feat], dim=1)
        output = self.output_head(combined)
        return output
    
    def training_step(self, batch, batch_idx):
        x_static, x_0M, x_0M_goutallier, y_2M = batch
        pred_2M = self.forward(x_static, x_0M, x_0M_goutallier)
        loss = F.mse_loss(pred_2M, y_2M)
        
        self.log("train/loss", loss, on_epoch=True, prog_bar=True)
        self.train_mse.update(pred_2M, y_2M)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x_static, x_0M, x_0M_goutallier, y_2M = batch
        pred_2M = self.forward(x_static, x_0M, x_0M_goutallier)
        loss = F.mse_loss(pred_2M, y_2M)
        
        self.log("val/loss", loss, on_epoch=True, prog_bar=True)
        self.val_mse.update(pred_2M, y_2M)
        return loss
    
    def on_train_epoch_end(self):
        self.log("train/mse", self.train_mse.compute())
        self.train_mse.reset()
    
    def on_validation_epoch_end(self):
        self.log("val/mse", self.val_mse.compute())
        self.val_mse.reset()
    
    def test_step(self, batch, batch_idx):
        x_static, x_0M, x_0M_goutallier, y_2M = batch
        pred_2M = self.forward(x_static, x_0M, x_0M_goutallier)
        loss = F.mse_loss(pred_2M, y_2M)
        
        self.log("test/loss", loss, on_epoch=True, prog_bar=True)
        self.test_mse.update(pred_2M, y_2M)
        return loss
    
    def on_test_epoch_end(self):
        self.log("test/mse", self.test_mse.compute())
        self.test_mse.reset()
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr, weight_decay=self.weight_decay)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.5, patience=5
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val/loss"
            }
        }

# Model 2 최적화 버전
class OptimizedSequentialMLP2(L.LightningModule):
    def __init__(self, trial, static_features, seq_0M_features, seq_2M_features, goutallier_0M_features, out_features_3M):
        super().__init__()
        
        self.static_encoder, static_out = create_encoder(
            trial, static_features, 'static_encoder', min_units=32, max_units=128
        )
        
        self.seq_0M_encoder, seq_0M_out = create_encoder(
            trial, seq_0M_features, 'seq_0M_encoder', min_units=64, max_units=256
        )
        
        self.seq_2M_encoder, seq_2M_out = create_encoder(
            trial, seq_2M_features, 'seq_2M_encoder', min_units=32, max_units=128
        )
        
        self.goutallier_0M_encoder, goutalier_out = create_encoder(
            trial, goutallier_0M_features, 'goutallier_0M_encoder', min_units=32, max_units=128
        )
        
        feat_dim = static_out + seq_0M_out + seq_2M_out + goutalier_out
        self.output_head = create_output_head(
            trial, feat_dim, out_features_3M, 'output_head', min_units=64, max_units=256
        )
        
        self.lr = trial.suggest_float('lr', 1e-7, 1e-4, log=True)
        self.weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-3, log=True)
        
        self.train_mse = MeanSquaredError()
        self.val_mse = MeanSquaredError()
        self.test_mse = MeanSquaredError()
        
    def forward(self, x_static, x_0M, x_2M, x_0M_goutallier):
        static_feat = self.static_encoder(x_static)
        seq_0M_feat = self.seq_0M_encoder(x_0M)
        seq_2M_feat = self.seq_2M_encoder(x_2M)
        goutallier_0M_feat = self.goutallier_0M_encoder(x_0M_goutallier)
        
        combined = torch.cat([static_feat, seq_0M_feat, seq_2M_feat, goutallier_0M_feat], dim=1)
        output = self.output_head(combined)
        return output
    
    def training_step(self, batch, batch_idx):
        x_static, x_0M, x_2M, x_0M_goutallier, y_3M = batch
        pred_3M = self.forward(x_static, x_0M, x_2M, x_0M_goutallier)
        loss = F.mse_loss(pred_3M, y_3M)
        
        self.log("train/loss", loss, on_epoch=True, prog_bar=True)
        self.train_mse.update(pred_3M, y_3M)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x_static, x_0M, x_2M, x_0M_goutallier, y_3M = batch
        pred_3M = self.forward(x_static, x_0M, x_2M, x_0M_goutallier)
        loss = F.mse_loss(pred_3M, y_3M)
        
        self.log("val/loss", loss, on_epoch=True, prog_bar=True)
        self.val_mse.update(pred_3M, y_3M)
        return loss
    
    def on_train_epoch_end(self):
        self.log("train/mse", self.train_mse.compute())
        self.train_mse.reset()
    
    def on_validation_epoch_end(self):
        self.log("val/mse", self.val_mse.compute())
        self.val_mse.reset()
    
    def test_step(self, batch, batch_idx):
        x_static, x_0M, x_2M, x_0M_goutallier, y_3M = batch
        pred_3M = self.forward(x_static, x_0M, x_2M, x_0M_goutallier)
        loss = F.mse_loss(pred_3M, y_3M)
        
        self.log("test/loss", loss, on_epoch=True, prog_bar=True)
        self.test_mse.update(pred_3M, y_3M)
        return loss
    
    def on_test_epoch_end(self):
        self.log("test/mse", self.test_mse.compute())
        self.test_mse.reset()
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr, weight_decay=self.weight_decay)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.5, patience=5
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val/loss"
            }
        }

# Model 3 최적화 버전
class OptimizedSequentialMLP3(L.LightningModule):
    def __init__(self, trial, static_features, seq_0M_features, seq_2M_features, seq_3M_features, goutallier_0M_features, out_features_4M):
        super().__init__()
        
        self.static_encoder, static_out = create_encoder(
            trial, static_features, 'static_encoder', min_units=32, max_units=128
        )
        
        self.seq_0M_encoder, seq_0M_out = create_encoder(
            trial, seq_0M_features, 'seq_0M_encoder', min_units=64, max_units=256
        )
        
        self.seq_2M_encoder, seq_2M_out = create_encoder(
            trial, seq_2M_features, 'seq_2M_encoder', min_units=32, max_units=128
        )
        
        self.seq_3M_encoder, seq_3M_out = create_encoder(
            trial, seq_3M_features, 'seq_3M_encoder', min_units=64, max_units=256
        )
        
        self.goutallier_0M_encoder, goutalier_out = create_encoder(
            trial, goutallier_0M_features, 'goutallier_0M_encoder', min_units=32, max_units=128
        )
        
        feat_dim = static_out + seq_0M_out + seq_2M_out + seq_3M_out + goutalier_out
        self.output_head = create_output_head(
            trial, feat_dim, out_features_4M, 'output_head', min_units=64, max_units=256
        )
        
        self.lr = trial.suggest_float('lr', 1e-7, 1e-4, log=True)
        self.weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-3, log=True)
        
        self.train_mse = MeanSquaredError()
        self.val_mse = MeanSquaredError()
        self.test_mse = MeanSquaredError()
        
    def forward(self, x_static, x_0M, x_2M, x_3M, x_0M_goutallier):
        static_feat = self.static_encoder(x_static)
        seq_0M_feat = self.seq_0M_encoder(x_0M)
        seq_2M_feat = self.seq_2M_encoder(x_2M)
        seq_3M_feat = self.seq_3M_encoder(x_3M)
        goutallier_0M_feat = self.goutallier_0M_encoder(x_0M_goutallier)
        
        combined = torch.cat([static_feat, seq_0M_feat, seq_2M_feat, seq_3M_feat, goutallier_0M_feat], dim=1)
        output = self.output_head(combined)
        return output
    
    def training_step(self, batch, batch_idx):
        x_static, x_0M, x_2M, x_3M, x_0M_goutallier, y_4M = batch
        pred_4M = self.forward(x_static, x_0M, x_2M, x_3M, x_0M_goutallier)
        loss = F.mse_loss(pred_4M, y_4M)
        
        self.log("train/loss", loss, on_epoch=True, prog_bar=True)
        self.train_mse.update(pred_4M, y_4M)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x_static, x_0M, x_2M, x_3M, x_0M_goutallier, y_4M = batch
        pred_4M = self.forward(x_static, x_0M, x_2M, x_3M, x_0M_goutallier)
        loss = F.mse_loss(pred_4M, y_4M)
        
        self.log("val/loss", loss, on_epoch=True, prog_bar=True)
        self.val_mse.update(pred_4M, y_4M)
        return loss
    
    def on_train_epoch_end(self):
        self.log("train/mse", self.train_mse.compute())
        self.train_mse.reset()
    
    def on_validation_epoch_end(self):
        self.log("val/mse", self.val_mse.compute())
        self.val_mse.reset()
    
    def test_step(self, batch, batch_idx):
        x_static, x_0M, x_2M, x_3M, x_0M_goutallier, y_4M = batch
        pred_4M = self.forward(x_static, x_0M, x_2M, x_3M, x_0M_goutallier)
        loss = F.mse_loss(pred_4M, y_4M)
        
        self.log("test/loss", loss, on_epoch=True, prog_bar=True)
        self.test_mse.update(pred_4M, y_4M)
        return loss
    
    def on_test_epoch_end(self):
        self.log("test/mse", self.test_mse.compute())
        self.test_mse.reset()
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr, weight_decay=self.weight_decay)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.5, patience=5
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val/loss"
            }
        }

# Model 4 최적화 버전 (분류 + 회귀)
class OptimizedSequentialMLP4(L.LightningModule):
    def __init__(self, trial, static_features, seq_0M_features, seq_2M_features, seq_3M_features, seq_4M_features, goutallier_0M_features, out_features_total, pos_weight=1.0):
        super().__init__()
        self.register_buffer('pos_weight', torch.tensor([pos_weight], dtype=torch.float32))
        
        # 회귀 손실 가중치 최적화
        self.reg_loss_weight = trial.suggest_float('reg_loss_weight', 0.1, 0.5, step=0.05)
        
        self.static_encoder, static_out = create_encoder(
            trial, static_features, 'static_encoder', min_units=32, max_units=128
        )
        
        self.seq_0M_encoder, seq_0M_out = create_encoder(
            trial, seq_0M_features, 'seq_0M_encoder', min_units=64, max_units=256
        )
        
        self.seq_2M_encoder, seq_2M_out = create_encoder(
            trial, seq_2M_features, 'seq_2M_encoder', min_units=32, max_units=128
        )
        
        self.seq_3M_encoder, seq_3M_out = create_encoder(
            trial, seq_3M_features, 'seq_3M_encoder', min_units=64, max_units=256
        )
        
        self.seq_4M_encoder, seq_4M_out = create_encoder(
            trial, seq_4M_features, 'seq_4M_encoder', min_units=64, max_units=256
        )
        
        self.goutallier_0M_encoder, goutalier_out = create_encoder(
            trial, goutallier_0M_features, 'goutallier_0M_encoder', min_units=32, max_units=128
        )
        
        feat_dim = static_out + seq_0M_out + seq_2M_out + seq_3M_out + seq_4M_out + goutalier_out
        
        # 분류 헤드
        self.clshead = create_output_head(
            trial, feat_dim, 1, 'clshead', min_units=64, max_units=256
        )
        
        # 회귀 헤드
        self.reghead = create_output_head(
            trial, feat_dim, out_features_total - 1, 'reghead', min_units=128, max_units=512
        )
        
        self.lr = trial.suggest_float('lr', 1e-7, 1e-4, log=True)
        self.weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-3, log=True)
        
        self.train_roc = BinaryAUROC()
        self.val_roc = BinaryAUROC()
        self.test_roc = BinaryAUROC()
        self.val_ap = BinaryAveragePrecision()
        self.test_ap = BinaryAveragePrecision()
        self.train_mse = MeanSquaredError()
        self.val_mse = MeanSquaredError()
        self.test_mse = MeanSquaredError()
        
    def forward(self, x_static, x_0M, x_2M, x_3M, x_4M, x_0M_goutallier):
        static_feat = self.static_encoder(x_static)
        seq_0M_feat = self.seq_0M_encoder(x_0M)
        seq_2M_feat = self.seq_2M_encoder(x_2M)
        seq_3M_feat = self.seq_3M_encoder(x_3M)
        seq_4M_feat = self.seq_4M_encoder(x_4M)
        goutallier_0M_feat = self.goutallier_0M_encoder(x_0M_goutallier)
        
        combined = torch.cat([static_feat, seq_0M_feat, seq_2M_feat, seq_3M_feat, seq_4M_feat, goutallier_0M_feat], dim=1)
        
        logits = self.clshead(combined)
        regs = self.reghead(combined)
        
        output = torch.cat([logits, regs], dim=1)
        return logits, regs, output
    
    def training_step(self, batch, batch_idx):
        x_static, x_0M, x_2M, x_3M, x_4M, x_0M_goutallier, y_combined = batch
        y_label = y_combined[:, seq_features_6M:seq_features_6M+1]
        y_reg = torch.cat([y_combined[:, :seq_features_6M], y_combined[:, seq_features_6M+1:]], dim=1)
        
        logits, regs, _ = self.forward(x_static, x_0M, x_2M, x_3M, x_4M, x_0M_goutallier)
        
        clf_loss = F.binary_cross_entropy_with_logits(logits, y_label, pos_weight=self.pos_weight)
        reg_loss = F.smooth_l1_loss(regs, y_reg)
        loss = clf_loss + self.reg_loss_weight * reg_loss
        
        self.log("train/loss", loss, on_epoch=True, prog_bar=True)
        self.log("train/clf_loss", clf_loss)
        self.log("train/reg_loss", reg_loss)
        
        probs = logits.sigmoid().flatten()
        targets = torch.clamp(y_label.flatten().to(torch.int), 0, 1)
        self.train_roc.update(probs, targets)
        self.train_mse.update(regs, y_reg)
        
        return loss
    
    def validation_step(self, batch, batch_idx):
        x_static, x_0M, x_2M, x_3M, x_4M, x_0M_goutallier, y_combined = batch
        y_label = y_combined[:, seq_features_6M:seq_features_6M+1]
        y_reg = torch.cat([y_combined[:, :seq_features_6M], y_combined[:, seq_features_6M+1:]], dim=1)
        
        logits, regs, _ = self.forward(x_static, x_0M, x_2M, x_3M, x_4M, x_0M_goutallier)
        
        clf_loss = F.binary_cross_entropy_with_logits(logits, y_label, pos_weight=self.pos_weight)
        reg_loss = F.smooth_l1_loss(regs, y_reg)
        loss = clf_loss + self.reg_loss_weight * reg_loss
        
        self.log("val/loss", loss, on_epoch=True, prog_bar=True)
        self.log("val/clf_loss", clf_loss)
        self.log("val/reg_loss", reg_loss)
        
        probs = logits.sigmoid().flatten()
        targets = torch.clamp(y_label.flatten().to(torch.int), 0, 1)
        self.val_roc.update(probs, targets)
        self.val_ap.update(probs, targets)
        self.val_mse.update(regs, y_reg)
        
        return loss
    
    def on_train_epoch_end(self):
        self.log("train/roc", self.train_roc.compute())
        self.log("train/mse", self.train_mse.compute())
        self.train_roc.reset()
        self.train_mse.reset()
    
    def on_validation_epoch_end(self):
        self.log("val/roc", self.val_roc.compute())
        self.log("val/ap", self.val_ap.compute())
        self.log("val/mse", self.val_mse.compute())
        self.val_roc.reset()
        self.val_ap.reset()
        self.val_mse.reset()
    
    def test_step(self, batch, batch_idx):
        x_static, x_0M, x_2M, x_3M, x_4M, x_0M_goutallier, y_combined = batch
        y_label = y_combined[:, seq_features_6M:seq_features_6M+1]
        y_reg = torch.cat([y_combined[:, :seq_features_6M], y_combined[:, seq_features_6M+1:]], dim=1)
        
        logits, regs, _ = self.forward(x_static, x_0M, x_2M, x_3M, x_4M, x_0M_goutallier)
        
        clf_loss = F.binary_cross_entropy_with_logits(logits, y_label, pos_weight=self.pos_weight)
        reg_loss = F.smooth_l1_loss(regs, y_reg)
        loss = clf_loss + self.reg_loss_weight * reg_loss
        
        self.log("test/loss", loss, on_epoch=True, prog_bar=True)
        self.log("test/clf_loss", clf_loss)
        self.log("test/reg_loss", reg_loss)
        
        probs = logits.sigmoid().flatten()
        targets = torch.clamp(y_label.flatten().to(torch.int), 0, 1)
        self.test_roc.update(probs, targets)
        self.test_ap.update(probs, targets)
        self.test_mse.update(regs, y_reg)
        
        return loss
    
    def on_test_epoch_end(self):
        self.log("test/roc", self.test_roc.compute())
        self.log("test/ap", self.test_ap.compute())
        self.log("test/mse", self.test_mse.compute())
        self.test_roc.reset()
        self.test_ap.reset()
        self.test_mse.reset()
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr, weight_decay=self.weight_decay)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='max', factor=0.5, patience=5
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val/roc"
            }
        }

print("최적화된 모델 클래스 정의 완료")


최적화된 모델 클래스 정의 완료


In [22]:
# 연결된 시계열 모델 클래스 (4개 모델을 순차적으로 실행)
class SequentialModel(nn.Module):
    """0M 입력만으로 6M 예측하는 연결된 시계열 모델"""
    def __init__(self, model1, model2, model3, model4):
        super().__init__()
        self.model1 = model1
        self.model2 = model2
        self.model3 = model3
        self.model4 = model4
        
        # 평가 모드로 설정
        self.model1.eval()
        self.model2.eval()
        self.model3.eval()
        self.model4.eval()
    
    @torch.no_grad()
    def forward(self, x_static, x_0M, x_0M_goutallier):
        """
        입력: static, 0M, 0M_goutallier
        출력: 6M features, y (logits), 6M_goutallier
        """
        # Model 1: static + 0M + 0M_goutallier → 2M
        pred_2M = self.model1(x_static, x_0M, x_0M_goutallier)
        
        # Model 2: static + 0M + 2M + 0M_goutallier → 3M
        pred_3M = self.model2(x_static, x_0M, pred_2M, x_0M_goutallier)
        
        # Model 3: static + 0M + 2M + 3M + 0M_goutallier → 4M
        pred_4M = self.model3(x_static, x_0M, pred_2M, pred_3M, x_0M_goutallier)
        
        # Model 4: static + 0M + 2M + 3M + 4M + 0M_goutallier → 6M + y + 6M_goutallier
        logits, regs, output = self.model4(x_static, x_0M, pred_2M, pred_3M, pred_4M, x_0M_goutallier)
        
        # 출력 분리: [6M (12) + y (1) + 6M_goutallier (8)]
        pred_6M = regs[:, :seq_features_6M]
        pred_y_logits = logits
        pred_6M_goutallier = regs[:, seq_features_6M:]
        
        return {
            'pred_2M': pred_2M,
            'pred_3M': pred_3M,
            'pred_4M': pred_4M,
            'pred_6M': pred_6M,
            'pred_y_logits': pred_y_logits,
            'pred_6M_goutallier': pred_6M_goutallier,
            'output': output
        }

In [23]:
# ============================================================================
# Optuna 최적화 함수들
# ============================================================================
CHECKPOINT_DIR = "../checkpoint"
OPTIMIZATION_MAX_EPOCH = 50
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

def optimize_model1(trial):
    """Model 1 최적화"""
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
    
    model = OptimizedSequentialMLP1(
        trial=trial,
        static_features=static_features,
        seq_0M_features=seq_features_0M,
        goutallier_0M_features=goutallier_features_0M,
        out_features_2M=seq_features_2M
    )
    
    trainloader = DataLoader(trainset_model1, batch_size=batch_size, shuffle=True, pin_memory=True)
    valloader = DataLoader(valset_model1, batch_size=batch_size)
    
    trainer = L.Trainer(
        max_epochs=OPTIMIZATION_MAX_EPOCH,
        gradient_clip_val=1.0,
        callbacks=[
            PyTorchLightningPruningCallback(trial, monitor="val/loss"),
            ModelCheckpoint(monitor='val/loss', mode='min', save_top_k=1, save_last=False, filename='optuna-model1-best', dirpath=CHECKPOINT_DIR),
            CleanupCheckpointCallback(checkpoint_dir=CHECKPOINT_DIR, filename_prefix='optuna-model1-best'),
            EarlyStopping(monitor='val/loss', mode='min', patience=10)
        ],
        enable_progress_bar=False,
        logger=False
    )
    
    trainer.fit(model, trainloader, valloader)
    return trainer.callback_metrics["val/loss"].item()

def optimize_model2(trial):
    """Model 2 최적화"""
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
    
    model = OptimizedSequentialMLP2(
        trial=trial,
        static_features=static_features,
        seq_0M_features=seq_features_0M,
        seq_2M_features=seq_features_2M,
        goutallier_0M_features=goutallier_features_0M,
        out_features_3M=seq_features_3M
    )
    
    trainloader = DataLoader(trainset_model2, batch_size=batch_size, shuffle=True, pin_memory=True)
    valloader = DataLoader(valset_model2, batch_size=batch_size)
    
    trainer = L.Trainer(
        max_epochs=OPTIMIZATION_MAX_EPOCH,
        gradient_clip_val=1.0,
        callbacks=[
            PyTorchLightningPruningCallback(trial, monitor="val/loss"),
            ModelCheckpoint(monitor='val/loss', mode='min', save_top_k=1, save_last=False, filename='optuna-model2-best', dirpath=CHECKPOINT_DIR),
            CleanupCheckpointCallback(checkpoint_dir=CHECKPOINT_DIR, filename_prefix='optuna-model2-best'),
            EarlyStopping(monitor='val/loss', mode='min', patience=10)
        ],
        enable_progress_bar=False,
        logger=False
    )
    
    trainer.fit(model, trainloader, valloader)
    return trainer.callback_metrics["val/loss"].item()

def optimize_model3(trial):
    """Model 3 최적화"""
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
    
    model = OptimizedSequentialMLP3(
        trial=trial,
        static_features=static_features,
        seq_0M_features=seq_features_0M,
        seq_2M_features=seq_features_2M,
        seq_3M_features=seq_features_3M,
        goutallier_0M_features=goutallier_features_0M,
        out_features_4M=seq_features_4M
    )
    
    trainloader = DataLoader(trainset_model3, batch_size=batch_size, shuffle=True, pin_memory=True)
    valloader = DataLoader(valset_model3, batch_size=batch_size)
    
    trainer = L.Trainer(
        max_epochs=OPTIMIZATION_MAX_EPOCH,
        gradient_clip_val=1.0,
        callbacks=[
            PyTorchLightningPruningCallback(trial, monitor="val/loss"),
            ModelCheckpoint(monitor='val/loss', mode='min', save_top_k=1, save_last=False, filename='optuna-model3-best', dirpath=CHECKPOINT_DIR),
            CleanupCheckpointCallback(checkpoint_dir=CHECKPOINT_DIR, filename_prefix='optuna-model3-best'),
            EarlyStopping(monitor='val/loss', mode='min', patience=10)
        ],
        enable_progress_bar=False,
        logger=False
    )
    
    trainer.fit(model, trainloader, valloader)
    return trainer.callback_metrics["val/loss"].item()

def optimize_model4(trial):
    """Model 4 최적화 (ROC AUC 최대화)"""
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
    
    model = OptimizedSequentialMLP4(
        trial=trial,
        static_features=static_features,
        seq_0M_features=seq_features_0M,
        seq_2M_features=seq_features_2M,
        seq_3M_features=seq_features_3M,
        seq_4M_features=seq_features_4M,
        goutallier_0M_features=goutallier_features_0M,
        out_features_total=seq_features_6M + 1 + goutallier_features_6M,
        pos_weight=cls_pos_weight
    )
    
    trainloader = DataLoader(trainset_model4, batch_size=batch_size, shuffle=True, pin_memory=True)
    valloader = DataLoader(valset_model4, batch_size=batch_size)
    
    trainer = L.Trainer(
        max_epochs=OPTIMIZATION_MAX_EPOCH,
        gradient_clip_val=1.0,
        callbacks=[
            PyTorchLightningPruningCallback(trial, monitor="val/roc"),
            ModelCheckpoint(monitor='val/roc', mode='max', save_top_k=1, save_last=False, filename='optuna-model4-best', dirpath=CHECKPOINT_DIR),
            CleanupCheckpointCallback(checkpoint_dir=CHECKPOINT_DIR, filename_prefix='optuna-model4-best'),
            EarlyStopping(monitor='val/roc', mode='max', patience=10)
        ],
        enable_progress_bar=False,
        logger=False
    )
    
    trainer.fit(model, trainloader, valloader)
    # ROC AUC를 최대화 (음수로 반환하여 최소화 문제로 변환)
    return -trainer.callback_metrics["val/roc"].item()

print("Optuna 최적화 함수 정의 완료")


Optuna 최적화 함수 정의 완료


### Optuna 실행 및 파라미터 최적화

In [24]:
TRIALS = 30  # 각 모델당 시도 횟수 (필요에 따라 조정 가능)

print("=" * 80)
print("NAS 기반 Optuna 하이퍼파라미터 최적화 시작")
print("=" * 80)

# Model 1 최적화
print("\n[Model 1] 최적화 시작...")
study1 = optuna.create_study(
    direction='minimize',
    study_name='sequential_mlp1_optimization',
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10)
)
study1.optimize(optimize_model1, n_trials=TRIALS, show_progress_bar=False)

print(f"\n[Model 1] 최적화 완료!")
print(f"최고 성능: {study1.best_value:.6f}")
print(f"최적 파라미터:")
for key, value in study1.best_params.items():
    print(f"  {key}: {value}")

# Model 2 최적화
print("\n[Model 2] 최적화 시작...")
study2 = optuna.create_study(
    direction='minimize',
    study_name='sequential_mlp2_optimization',
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10)
)
study2.optimize(optimize_model2, n_trials=TRIALS, show_progress_bar=False)

print(f"\n[Model 2] 최적화 완료!")
print(f"최고 성능: {study2.best_value:.6f}")
print(f"최적 파라미터:")
for key, value in study2.best_params.items():
    print(f"  {key}: {value}")

# Model 3 최적화
print("\n[Model 3] 최적화 시작...")
study3 = optuna.create_study(
    direction='minimize',
    study_name='sequential_mlp3_optimization',
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10)
)
study3.optimize(optimize_model3, n_trials=TRIALS, show_progress_bar=False)

print(f"\n[Model 3] 최적화 완료!")
print(f"최고 성능: {study3.best_value:.6f}")
print(f"최적 파라미터:")
for key, value in study3.best_params.items():
    print(f"  {key}: {value}")

# Model 4 최적화
print("\n[Model 4] 최적화 시작...")
study4 = optuna.create_study(
    direction='minimize',  # 음수 ROC AUC를 최소화 = ROC AUC 최대화
    study_name='sequential_mlp4_optimization',
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10)
)
study4.optimize(optimize_model4, n_trials=TRIALS, show_progress_bar=False)

print(f"\n[Model 4] 최적화 완료!")
print(f"최고 성능 (음수 ROC AUC): {study4.best_value:.6f}")
print(f"실제 최고 ROC AUC: {-study4.best_value:.6f}")
print(f"최적 파라미터:")
for key, value in study4.best_params.items():
    print(f"  {key}: {value}")

print("\n" + "=" * 80)
print("모든 모델 최적화 완료!")
print("=" * 80)

[I 2025-11-10 15:11:20,705] A new study created in memory with name: sequential_mlp1_optimization
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
You are using a CUDA device ('NVIDIA GeForce RTX 4060 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


NAS 기반 Optuna 하이퍼파라미터 최적화 시작

[Model 1] 최적화 시작...


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                  | Type             | Params | Mode 
-------------------------------------------------------------------
0 | static_encoder        | Sequential       | 15.5 K | train
1 | seq_0M_encoder        | Sequential       | 200 K  | train
2 | goutallier_0M_encoder | Sequential       | 50.8 K | train
3 | output_head           | Sequential       | 639 K  | train
4 | train_mse             | MeanSquaredError | 0      | train
5 | val_mse               | MeanSquaredError | 0      | train
6 | test_mse              | MeanSquaredError | 0      | train
-------------------------------------------------------------------
906 K     Trainable params
0         Non-trainable params
906 K     Total params
3.624     Total estimated model params size (MB)
83        Modules in train mode
0         Modules in eval mode
c:\Users\mulso\anaconda3\envs\arcr\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:433: The 'val_dataloader

[체크포인트 정리] 1개의 체크포인트 제거 완료. 최신 체크포인트만 유지: optuna-model1-best-v1.ckpt


[I 2025-11-10 15:12:23,799] Trial 2 finished with value: 1.0721687078475952 and parameters: {'batch_size': 64, 'static_encoder_n_layers': 2, 'static_encoder_units_0': 192, 'static_encoder_dropout_0': 0.2, 'static_encoder_batch_norm_0': True, 'static_encoder_units_1': 128, 'static_encoder_dropout_1': 0.05, 'static_encoder_batch_norm_1': False, 'seq_0M_encoder_n_layers': 4, 'seq_0M_encoder_units_0': 64, 'seq_0M_encoder_dropout_0': 0.4, 'seq_0M_encoder_batch_norm_0': True, 'seq_0M_encoder_units_1': 256, 'seq_0M_encoder_dropout_1': 0.45, 'seq_0M_encoder_batch_norm_1': True, 'seq_0M_encoder_units_2': 160, 'seq_0M_encoder_dropout_2': 0.0, 'seq_0M_encoder_batch_norm_2': False, 'seq_0M_encoder_units_3': 128, 'seq_0M_encoder_dropout_3': 0.25, 'seq_0M_encoder_batch_norm_3': True, 'goutallier_0M_encoder_n_layers': 2, 'goutallier_0M_encoder_units_0': 128, 'goutallier_0M_encoder_dropout_0': 0.4, 'goutallier_0M_encoder_batch_norm_0': False, 'goutallier_0M_encoder_units_1': 96, 'goutallier_0M_encoder

...

[체크포인트 정리] 2개의 체크포인트 제거 완료. 최신 체크포인트만 유지: optuna-model4-best-v1.ckpt


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                  | Type                   | Params | Mode 
--------------------------------------------------------------------------
0  | static_encoder        | Sequential             | 73.5 K | train
1  | seq_0M_encoder        | Sequential             | 140 K  | train
2  | seq_2M_encoder        | Sequential             | 69.9 K | train
3  | seq_3M_encoder        | Sequential             | 58.0 K | train
4  | seq_4M_encoder        | Sequential             | 139 K  | train
5  | goutallier_0M_encoder | Sequential             | 32.6 K | train
6  | clshead               | Sequential             | 705    | train
7  | reghead               | Sequential             | 325 K  | train
8  | train_roc             | BinaryAUROC            | 0      | train
9  | val_roc               | BinaryAUROC            | 0      | train
10 | test_roc              | BinaryAUROC   

[체크포인트 정리] 2개의 체크포인트 제거 완료. 최신 체크포인트만 유지: optuna-model4-best-v2.ckpt


TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                  | Type                   | Params | Mode 
--------------------------------------------------------------------------
0  | static_encoder        | Sequential             | 23.4 K | train
1  | seq_0M_encoder        | Sequential             | 144 K  | train
2  | seq_2M_encoder        | Sequential             | 4.8 K  | train
3  | seq_3M_encoder        | Sequential             | 10.9 K | train
4  | seq_4M_encoder        | Sequential             | 20.4 K | train
5  | goutallier_0M_encoder | Sequential             | 7.4 K  | train
6  | clshead               | Sequential             | 112 K  | train
7  | reghead               | Sequential             | 9.6 K  | train
8  | train_roc             | BinaryAUROC            | 0      | train
9  | val_roc               | BinaryAUROC            | 0      | train
10 | test_roc              | BinaryAUROC            | 0      | train
11 | val_ap  

[체크포인트 정리] 1개의 체크포인트 제거 완료. 최신 체크포인트만 유지: optuna-model4-best.ckpt

[Model 4] 최적화 완료!
최고 성능 (음수 ROC AUC): -0.778209
실제 최고 ROC AUC: 0.778209
최적 파라미터:
  batch_size: 32
  reg_loss_weight: 0.25
  static_encoder_n_layers: 8
  static_encoder_units_0: 128
  static_encoder_dropout_0: 0.35000000000000003
  static_encoder_batch_norm_0: False
  static_encoder_units_1: 64
  static_encoder_dropout_1: 0.2
  static_encoder_batch_norm_1: False
  static_encoder_units_2: 64
  static_encoder_dropout_2: 0.1
  static_encoder_batch_norm_2: True
  static_encoder_units_3: 32
  static_encoder_dropout_3: 0.15000000000000002
  static_encoder_batch_norm_3: False
  static_encoder_units_4: 64
  static_encoder_dropout_4: 0.1
  static_encoder_batch_norm_4: True
  static_encoder_units_5: 128
  static_encoder_dropout_5: 0.4
  static_encoder_batch_norm_5: False
  static_encoder_units_6: 128
  static_encoder_dropout_6: 0.30000000000000004
  static_encoder_batch_norm_6: True
  static_encoder_units_7: 96
  static_encoder_dr

### 개별 모델 및 Sequential 모델 성능 평가 함수 정의

In [25]:
@torch.no_grad()
def evaluate_individual_model(model, testloader, model_name):
    """개별 모델의 성능 평가"""
    model.eval()
    
    all_preds = []
    all_targets = []
    
    for batch in testloader:
        if model_name == "Model1":
            x_static, x_0M, x_0M_goutallier, y_2M = batch
            pred = model(x_static, x_0M, x_0M_goutallier)
            all_preds.append(pred)
            all_targets.append(y_2M)
        elif model_name == "Model2":
            x_static, x_0M, x_2M, x_0M_goutallier, y_3M = batch
            pred = model(x_static, x_0M, x_2M, x_0M_goutallier)
            all_preds.append(pred)
            all_targets.append(y_3M)
        elif model_name == "Model3":
            x_static, x_0M, x_2M, x_3M, x_0M_goutallier, y_4M = batch
            pred = model(x_static, x_0M, x_2M, x_3M, x_0M_goutallier)
            all_preds.append(pred)
            all_targets.append(y_4M)
        elif model_name == "Model4":
            x_static, x_0M, x_2M, x_3M, x_4M, x_0M_goutallier, y_combined = batch
            logits, regs, _ = model(x_static, x_0M, x_2M, x_3M, x_4M, x_0M_goutallier)
            y_label = y_combined[:, seq_features_6M:seq_features_6M+1]
            y_reg = torch.cat([y_combined[:, :seq_features_6M], y_combined[:, seq_features_6M+1:]], dim=1)
            
            all_preds.append({'logits': logits, 'regs': regs})
            all_targets.append({'label': y_label, 'reg': y_reg})
    
    if model_name == "Model4":
        # 분류 및 회귀 성능 계산
        all_logits = torch.cat([p['logits'] for p in all_preds], dim=0)
        all_regs = torch.cat([p['regs'] for p in all_preds], dim=0)
        all_labels = torch.cat([t['label'] for t in all_targets], dim=0)
        all_reg_targets = torch.cat([t['reg'] for t in all_targets], dim=0)
        
        mse = F.mse_loss(all_regs, all_reg_targets).item()
        probs = all_logits.sigmoid().flatten()
        labels_int = all_labels.flatten().to(torch.int)
        
        # 분류 성능 계산
        pred_labels = (probs > 0.5).int().cpu().numpy()
        labels_np = labels_int.cpu().numpy()
        
        roc_auc = roc_auc_score(labels_np, probs.cpu().numpy())
        ap = average_precision_score(labels_np, probs.cpu().numpy())
        accuracy = accuracy_score(labels_np, pred_labels)
        
        return {
            'mse': mse,
            'roc_auc': roc_auc,
            'ap': ap,
            'accuracy': accuracy
        }
    else:
        # 회귀 성능만 계산
        all_preds = torch.cat(all_preds, dim=0)
        all_targets = torch.cat(all_targets, dim=0)
        mse = F.mse_loss(all_preds, all_targets).item()
        mae = F.l1_loss(all_preds, all_targets).item()
        
        return {
            'mse': mse,
            'mae': mae
        }

@torch.no_grad()
def evaluate_sequential_model(sequential_model, testset_model1):
    """Sequential 모델의 성능 평가 (0M 입력만으로 전체 시계열 예측)"""
    sequential_model.eval()
    
    all_pred_2M = []
    all_pred_3M = []
    all_pred_4M = []
    all_pred_6M = []
    all_pred_y_logits = []
    all_pred_6M_goutallier = []
    
    all_true_2M = []
    all_true_3M = []
    all_true_4M = []
    all_true_6M = []
    all_true_y = []
    all_true_6M_goutallier = []
    
    for i in range(len(testset_model1)):
        x_static, x_0M, x_0M_goutallier, y_2M = testset_model1[i]
        x_static = x_static.unsqueeze(0)
        x_0M = x_0M.unsqueeze(0)
        x_0M_goutallier = x_0M_goutallier.unsqueeze(0)
        
        # Sequential 모델 예측
        predictions = sequential_model(x_static, x_0M, x_0M_goutallier)
        
        all_pred_2M.append(predictions['pred_2M'])
        all_pred_3M.append(predictions['pred_3M'])
        all_pred_4M.append(predictions['pred_4M'])
        all_pred_6M.append(predictions['pred_6M'])
        all_pred_y_logits.append(predictions['pred_y_logits'])
        all_pred_6M_goutallier.append(predictions['pred_6M_goutallier'])
        
        # 실제 값 추출
        _, _, _, _, y_3M = testset_model2[i]
        _, _, _, _, _, y_4M = testset_model3[i]
        _, _, _, _, _, _, y_combined = testset_model4[i]
        
        y_6M = y_combined[:seq_features_6M]
        y_label = y_combined[seq_features_6M:seq_features_6M+1]
        y_6M_goutallier = y_combined[seq_features_6M+1:]
        
        all_true_2M.append(y_2M)
        all_true_3M.append(y_3M)
        all_true_4M.append(y_4M)
        all_true_6M.append(y_6M)
        all_true_y.append(y_label)
        all_true_6M_goutallier.append(y_6M_goutallier)
    
    # 텐서로 변환
    pred_2M = torch.cat(all_pred_2M, dim=0)
    pred_3M = torch.cat(all_pred_3M, dim=0)
    pred_4M = torch.cat(all_pred_4M, dim=0)
    pred_6M = torch.cat(all_pred_6M, dim=0)
    pred_y_logits = torch.cat(all_pred_y_logits, dim=0)
    pred_6M_goutallier = torch.cat(all_pred_6M_goutallier, dim=0)
    
    true_2M = torch.stack(all_true_2M)
    true_3M = torch.stack(all_true_3M)
    true_4M = torch.stack(all_true_4M)
    true_6M = torch.stack(all_true_6M)
    true_y = torch.stack(all_true_y)
    true_6M_goutallier = torch.stack(all_true_6M_goutallier)
    
    # 성능 계산
    mse_2M = F.mse_loss(pred_2M, true_2M).item()
    mse_3M = F.mse_loss(pred_3M, true_3M).item()
    mse_4M = F.mse_loss(pred_4M, true_4M).item()
    mse_6M = F.mse_loss(pred_6M, true_6M).item()
    mse_6M_goutallier = F.mse_loss(pred_6M_goutallier, true_6M_goutallier).item()
    
    mae_2M = F.l1_loss(pred_2M, true_2M).item()
    mae_3M = F.l1_loss(pred_3M, true_3M).item()
    mae_4M = F.l1_loss(pred_4M, true_4M).item()
    mae_6M = F.l1_loss(pred_6M, true_6M).item()
    mae_6M_goutallier = F.l1_loss(pred_6M_goutallier, true_6M_goutallier).item()
    
    # 분류 성능
    pred_y_probs = pred_y_logits.sigmoid().flatten()
    true_y_int = true_y.flatten().to(torch.int)
    
    # 분류 예측 및 성능 계산
    pred_labels = (pred_y_probs > 0.5).int().cpu().numpy()
    labels_np = true_y_int.cpu().numpy()
    
    roc_auc = roc_auc_score(labels_np, pred_y_probs.cpu().numpy())
    ap = average_precision_score(labels_np, pred_y_probs.cpu().numpy())
    accuracy = accuracy_score(labels_np, pred_labels)
    
    return {
        'mse_2M': mse_2M,
        'mse_3M': mse_3M,
        'mse_4M': mse_4M,
        'mse_6M': mse_6M,
        'mse_6M_goutallier': mse_6M_goutallier,
        'mae_2M': mae_2M,
        'mae_3M': mae_3M,
        'mae_4M': mae_4M,
        'mae_6M': mae_6M,
        'mae_6M_goutallier': mae_6M_goutallier,
        'roc_auc': roc_auc,
        'ap': ap,
        'accuracy': accuracy
    }

print("성능 평가 함수 정의 완료")


성능 평가 함수 정의 완료


### 최적화된 파라미터로 최종 모델 생성 및 학습

In [26]:
MAX_EPOCH = 300

def create_final_model_from_study(study, model_class, model_kwargs):
    """최적화된 파라미터로 최종 모델 생성"""
    best_trial = study.best_trial
    fixed_trial = FixedTrial(best_trial.params)
    model = model_class(trial=fixed_trial, **model_kwargs)
    return model, best_trial.params

# 최적화된 모델 생성
print("\n" + "=" * 80)
print("최적화된 파라미터로 최종 모델 생성")
print("=" * 80)

# Model 1 최종 모델
print("\n[Model 1] 최종 모델 생성 중...")
final_model1, best_params1 = create_final_model_from_study(
    study1,
    OptimizedSequentialMLP1,
    {
        'static_features': static_features,
        'seq_0M_features': seq_features_0M,
        'goutallier_0M_features': goutallier_features_0M,
        'out_features_2M': seq_features_2M
    }
)

# Model 2 최종 모델
print("[Model 2] 최종 모델 생성 중...")
final_model2, best_params2 = create_final_model_from_study(
    study2,
    OptimizedSequentialMLP2,
    {
        'static_features': static_features,
        'seq_0M_features': seq_features_0M,
        'seq_2M_features': seq_features_2M,
        'goutallier_0M_features': goutallier_features_0M,
        'out_features_3M': seq_features_3M
    }
)

# Model 3 최종 모델
print("[Model 3] 최종 모델 생성 중...")
final_model3, best_params3 = create_final_model_from_study(
    study3,
    OptimizedSequentialMLP3,
    {
        'static_features': static_features,
        'seq_0M_features': seq_features_0M,
        'seq_2M_features': seq_features_2M,
        'seq_3M_features': seq_features_3M,
        'goutallier_0M_features': goutallier_features_0M,
        'out_features_4M': seq_features_4M
    }
)

# Model 4 최종 모델
print("[Model 4] 최종 모델 생성 중...")
final_model4, best_params4 = create_final_model_from_study(
    study4,
    OptimizedSequentialMLP4,
    {
        'static_features': static_features,
        'seq_0M_features': seq_features_0M,
        'seq_2M_features': seq_features_2M,
        'seq_3M_features': seq_features_3M,
        'seq_4M_features': seq_features_4M,
        'goutallier_0M_features': goutallier_features_0M,
        'out_features_total': seq_features_6M + 1 + goutallier_features_6M,
        'pos_weight': cls_pos_weight
    }
)

print("\n최종 모델 생성 완료!")

# 최종 모델 학습
print("\n" + "=" * 80)
print("최종 모델 학습 시작")
print("=" * 80)

# 최적 배치 크기 추출
batch_size1 = best_params1.get('batch_size', 64)
batch_size2 = best_params2.get('batch_size', 64)
batch_size3 = best_params3.get('batch_size', 64)
batch_size4 = best_params4.get('batch_size', 64)

# Model 1 학습
print("\n[Model 1] 학습 시작...")
trainloader1_final = DataLoader(trainset_model1, batch_size=batch_size1, shuffle=True, pin_memory=True)
valloader1_final = DataLoader(valset_model1, batch_size=batch_size1)
testloader1_final = DataLoader(testset_model1, batch_size=batch_size1)

trainer1_final = L.Trainer(
    max_epochs=MAX_EPOCH,
    gradient_clip_val=1.0,
    callbacks=[
        ModelCheckpoint(monitor='val/loss', mode='min', save_top_k=1, save_last=False, filename='final-model1-best', dirpath=CHECKPOINT_DIR),
        CleanupCheckpointCallback(checkpoint_dir=CHECKPOINT_DIR, filename_prefix='final-model1-best'),
        EarlyStopping(monitor='val/loss', mode='min', patience=15)
    ]
)

trainer1_final.fit(final_model1, trainloader1_final, valloader1_final)
test_result1_final = trainer1_final.test(final_model1, testloader1_final)
print(f"[Model 1] 학습 완료 - Test MSE: {test_result1_final[0]['test/mse']:.6f}")

# Model 2 학습
print("\n[Model 2] 학습 시작...")
trainloader2_final = DataLoader(trainset_model2, batch_size=batch_size2, shuffle=True, pin_memory=True)
valloader2_final = DataLoader(valset_model2, batch_size=batch_size2)
testloader2_final = DataLoader(testset_model2, batch_size=batch_size2)

trainer2_final = L.Trainer(
    max_epochs=MAX_EPOCH,
    gradient_clip_val=1.0,
    callbacks=[
        ModelCheckpoint(monitor='val/loss', mode='min', save_top_k=1, save_last=False, filename='final-model2-best', dirpath=CHECKPOINT_DIR),
        CleanupCheckpointCallback(checkpoint_dir=CHECKPOINT_DIR, filename_prefix='final-model2-best'),
        EarlyStopping(monitor='val/loss', mode='min', patience=15)
    ]
)

trainer2_final.fit(final_model2, trainloader2_final, valloader2_final)
test_result2_final = trainer2_final.test(final_model2, testloader2_final)
print(f"[Model 2] 학습 완료 - Test MSE: {test_result2_final[0]['test/mse']:.6f}")

# Model 3 학습
print("\n[Model 3] 학습 시작...")
trainloader3_final = DataLoader(trainset_model3, batch_size=batch_size3, shuffle=True, pin_memory=True)
valloader3_final = DataLoader(valset_model3, batch_size=batch_size3)
testloader3_final = DataLoader(testset_model3, batch_size=batch_size3)

trainer3_final = L.Trainer(
    max_epochs=MAX_EPOCH,
    gradient_clip_val=1.0,
    callbacks=[
        ModelCheckpoint(monitor='val/loss', mode='min', save_top_k=1, save_last=False, filename='final-model3-best', dirpath=CHECKPOINT_DIR),
        CleanupCheckpointCallback(checkpoint_dir=CHECKPOINT_DIR, filename_prefix='final-model3-best'),
        EarlyStopping(monitor='val/loss', mode='min', patience=15)
    ]
)

trainer3_final.fit(final_model3, trainloader3_final, valloader3_final)
test_result3_final = trainer3_final.test(final_model3, testloader3_final)
print(f"[Model 3] 학습 완료 - Test MSE: {test_result3_final[0]['test/mse']:.6f}")

# Model 4 학습
print("\n[Model 4] 학습 시작...")
trainloader4_final = DataLoader(trainset_model4, batch_size=batch_size4, shuffle=True, pin_memory=True)
valloader4_final = DataLoader(valset_model4, batch_size=batch_size4)
testloader4_final = DataLoader(testset_model4, batch_size=batch_size4)

trainer4_final = L.Trainer(
    max_epochs=MAX_EPOCH,
    gradient_clip_val=1.0,
    callbacks=[
        ModelCheckpoint(monitor='val/roc', mode='max', save_top_k=1, save_last=False, filename='final-model4-best', dirpath=CHECKPOINT_DIR),
        CleanupCheckpointCallback(checkpoint_dir=CHECKPOINT_DIR, filename_prefix='final-model4-best'),
        EarlyStopping(monitor='val/roc', mode='max', patience=15)
    ]
)

trainer4_final.fit(final_model4, trainloader4_final, valloader4_final)
test_result4_final = trainer4_final.test(final_model4, testloader4_final)
print(f"[Model 4] 학습 완료 - Test ROC: {test_result4_final[0]['test/roc']:.6f}, Test AP: {test_result4_final[0]['test/ap']:.6f}")

print("\n" + "=" * 80)
print("모든 최종 모델 학습 완료!")
print("=" * 80)



최적화된 파라미터로 최종 모델 생성

[Model 1] 최종 모델 생성 중...
[Model 2] 최종 모델 생성 중...
[Model 3] 최종 모델 생성 중...
[Model 4] 최종 모델 생성 중...

최종 모델 생성 완료!

최종 모델 학습 시작

[Model 1] 학습 시작...


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
c:\Users\mulso\anaconda3\envs\arcr\lib\site-packages\lightning\pytorch\trainer\connectors\logger_connector\logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                  | Type             | Params | Mode 
-------------------------------------------------------------------
0 | static_encoder        | Sequential       | 1.8 K  | train
1 | seq_0M_encoder        | Sequential       | 2.4 K  | train
2 | goutallier_0M_encoder | Sequential       | 36.8 K | train
3 | output_head         

Epoch 77: 100%|██████████| 225/225 [00:02<00:00, 86.13it/s, v_num=6, train/loss_step=1.770, val/loss=1.030, train/loss_epoch=0.847] 


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\mulso\anaconda3\envs\arcr\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:433: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 4/4 [00:00<00:00, 113.88it/s]

...

Testing DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 90.35it/s] 
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test/loss           1.3424181938171387
        test/mse            1.3424181938171387
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
[Model 3] 학습 완료 - Test MSE: 1.342418

[Model 4] 학습 시작...


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                  | Type                   | Params | Mode 
--------------------------------------------------------------------------
0  | static_encoder        | Sequential             | 58.6 K | train
1  | seq_0M_encoder        | Sequential             | 53.6 K | train
2  | seq_2M_encoder        | Sequential             | 53.7 K | train
3  | seq_3M_encoder        | Sequential             | 83.2 K | train
4  | seq_4M_encoder        | Sequential             | 99.3 K | train
5  | goutallier_0M_encoder | Sequential             | 43.5 K | train
6  | clshead               | Sequential             | 929    | train
7  | reghead               | Sequential             | 243 K  | train
8  | train_roc             | BinaryAUROC            | 0      | train
9  | val_roc               | BinaryAUROC            | 0      | train
10 | test_roc              | BinaryAUROC   

Epoch 42: 100%|██████████| 225/225 [00:04<00:00, 47.70it/s, v_num=9, train/loss_step=0.672, val/loss=0.693, train/loss_epoch=0.717]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 4/4 [00:00<00:00, 69.83it/s] 
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         test/ap            0.7812328338623047
      test/clf_loss         0.6064221262931824
        test/loss           0.7100300788879395
        test/mse             1.383682370185852
      test/reg_loss         0.41443172097206116
        test/roc            0.7716000080108643
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
[Model 4] 학습 완료 - Test ROC: 0.771600, Test AP: 0.781233

모든 최종 모델 학습 완료!


### Sequential 모델 연결 및 성능 평가

In [27]:
print("\n" + "=" * 80)
print("최적화된 Sequential 모델 생성 및 성능 평가")
print("=" * 80)

# Sequential 모델 생성
sequential_model = SequentialModel(final_model1, final_model2, final_model3, final_model4)
print("Sequential 모델 생성 완료")

# Test DataLoader 생성
batch_size = 64
testloader1 = DataLoader(testset_model1, batch_size=batch_size)
testloader2 = DataLoader(testset_model2, batch_size=batch_size)
testloader3 = DataLoader(testset_model3, batch_size=batch_size)
testloader4 = DataLoader(testset_model4, batch_size=batch_size)

# 개별 모델 성능 평가
print("\n" + "-" * 80)
print("개별 모델 성능 평가")
print("-" * 80)

print("\n[Model 1] 평가 중...")
model1_results = evaluate_individual_model(final_model1, testloader1, "Model1")
print(f"  MSE: {model1_results['mse']:.6f}")
print(f"  MAE: {model1_results['mae']:.6f}")

print("\n[Model 2] 평가 중...")
model2_results = evaluate_individual_model(final_model2, testloader2, "Model2")
print(f"  MSE: {model2_results['mse']:.6f}")
print(f"  MAE: {model2_results['mae']:.6f}")

print("\n[Model 3] 평가 중...")
model3_results = evaluate_individual_model(final_model3, testloader3, "Model3")
print(f"  MSE: {model3_results['mse']:.6f}")
print(f"  MAE: {model3_results['mae']:.6f}")

print("\n[Model 4] 평가 중...")
model4_results = evaluate_individual_model(final_model4, testloader4, "Model4")
print(f"  MSE: {model4_results['mse']:.6f}")
print(f"  ROC AUC: {model4_results['roc_auc']:.6f}")
print(f"  AP: {model4_results['ap']:.6f}")
print(f"  Accuracy: {model4_results['accuracy']:.6f}")

# Sequential 모델 성능 평가
print("\n" + "-" * 80)
print("Sequential 모델 성능 평가 (0M 입력만으로 전체 시계열 예측)")
print("-" * 80)

sequential_results = evaluate_sequential_model(sequential_model, testset_model1)

print("\n[회귀 성능 - MSE]")
print(f"  2M 예측 MSE: {sequential_results['mse_2M']:.6f}")
print(f"  3M 예측 MSE: {sequential_results['mse_3M']:.6f}")
print(f"  4M 예측 MSE: {sequential_results['mse_4M']:.6f}")
print(f"  6M 예측 MSE: {sequential_results['mse_6M']:.6f}")
print(f"  6M Goutallier 예측 MSE: {sequential_results['mse_6M_goutallier']:.6f}")

print("\n[회귀 성능 - MAE]")
print(f"  2M 예측 MAE: {sequential_results['mae_2M']:.6f}")
print(f"  3M 예측 MAE: {sequential_results['mae_3M']:.6f}")
print(f"  4M 예측 MAE: {sequential_results['mae_4M']:.6f}")
print(f"  6M 예측 MAE: {sequential_results['mae_6M']:.6f}")
print(f"  6M Goutallier 예측 MAE: {sequential_results['mae_6M_goutallier']:.6f}")

print("\n[분류 성능]")
print(f"  ROC AUC: {sequential_results['roc_auc']:.6f}")
print(f"  AP (Average Precision): {sequential_results['ap']:.6f}")
print(f"  Accuracy: {sequential_results['accuracy']:.6f}")

# 성능 요약
print("\n" + "=" * 80)
print("성능 평가 요약")
print("=" * 80)
print("\n개별 모델 성능:")
print(f"  Model 1 (2M 예측): MSE={model1_results['mse']:.6f}, MAE={model1_results['mae']:.6f}")
print(f"  Model 2 (3M 예측): MSE={model2_results['mse']:.6f}, MAE={model2_results['mae']:.6f}")
print(f"  Model 3 (4M 예측): MSE={model3_results['mse']:.6f}, MAE={model3_results['mae']:.6f}")
print(f"  Model 4 (6M+분류): MSE={model4_results['mse']:.6f}, ROC={model4_results['roc_auc']:.6f}, AP={model4_results['ap']:.6f}, Accuracy={model4_results['accuracy']:.6f}")
print("\nSequential 모델 성능:")
print(f"  전체 파이프라인: ROC={sequential_results['roc_auc']:.6f}, AP={sequential_results['ap']:.6f}, Accuracy={sequential_results['accuracy']:.6f}")
print(f"  중간 단계 예측: 2M MSE={sequential_results['mse_2M']:.6f}, 3M MSE={sequential_results['mse_3M']:.6f}, 4M MSE={sequential_results['mse_4M']:.6f}, 6M MSE={sequential_results['mse_6M']:.6f}")

print("\n" + "=" * 80)
print("모든 평가 완료!")
print("=" * 80)



최적화된 Sequential 모델 생성 및 성능 평가
Sequential 모델 생성 완료

--------------------------------------------------------------------------------
개별 모델 성능 평가
--------------------------------------------------------------------------------

[Model 1] 평가 중...
  MSE: 0.946855
  MAE: 0.771914

[Model 2] 평가 중...
  MSE: 0.914042
  MAE: 0.696520

[Model 3] 평가 중...
  MSE: 1.342418
  MAE: 0.780300

[Model 4] 평가 중...
  MSE: 1.383682
  ROC AUC: 0.771600
  AP: 0.781233
  Accuracy: 0.700000

--------------------------------------------------------------------------------
Sequential 모델 성능 평가 (0M 입력만으로 전체 시계열 예측)
--------------------------------------------------------------------------------

[회귀 성능 - MSE]
  2M 예측 MSE: 0.946855
  3M 예측 MSE: 1.032056
  4M 예측 MSE: 1.618467
  6M 예측 MSE: 1.688618
  6M Goutallier 예측 MSE: 1.058361

[회귀 성능 - MAE]
  2M 예측 MAE: 0.771914
  3M 예측 MAE: 0.753499
  4M 예측 MAE: 0.891888
  6M 예측 MAE: 0.923617
  6M Goutallier 예측 MAE: 0.446009

[분류 성능]
  ROC AUC: 0.570000
  AP (Average Precision):

### Sequential 모델 파인튜닝


In [28]:
# 파인튜닝용 데이터셋 생성 함수
def get_dataset_finetuning(split):
    """Sequential 모델 파인튜닝용 데이터셋
    입력: static + 0M + 0M_goutallier
    출력: label_column (바이너리) + output_columns (회귀)
    """
    assert split in ["train", "val", "test"]
    
    X_file_name = f"{DATA_PATH}/X_{split}.csv"
    y_file_name = f"{DATA_PATH}/y_{split}.csv"
    
    X = pd.read_csv(X_file_name)
    y = pd.read_csv(y_file_name)
    
    # 입력: static + 0M + 0M_goutallier
    X_static = torch.tensor(X[static_columns].to_numpy(), dtype=torch.float32)
    X_0M = torch.tensor(X[seq_columns_0M].to_numpy(), dtype=torch.float32)
    X_0M_goutallier = torch.tensor(X[goutallier_columns_0M + goutallier_columns_0M_missing].to_numpy(), dtype=torch.float32)
    
    # 출력: label_column (바이너리) + output_columns (회귀)
    y_label = torch.tensor(y[label_column].to_numpy(), dtype=torch.float32).unsqueeze(1)
    y_output = torch.tensor(X[output_columns].to_numpy(), dtype=torch.float32)
    
    return TensorDataset(X_static, X_0M, X_0M_goutallier, y_label, y_output)

# 파인튜닝용 데이터셋 생성
trainset_finetuning = get_dataset_finetuning("train")
valset_finetuning = get_dataset_finetuning("val")
testset_finetuning = get_dataset_finetuning("test")

print(f"파인튜닝 데이터셋 생성 완료")
print(f"Train: {len(trainset_finetuning)}, Val: {len(valset_finetuning)}, Test: {len(testset_finetuning)}")


파인튜닝 데이터셋 생성 완료
Train: 7178, Val: 647, Test: 100


In [29]:
# 파인튜닝 가능한 Sequential 모델 클래스
class SequentialFineTuningModel(L.LightningModule):
    """Sequential 모델을 파인튜닝하기 위한 Lightning 모듈
    입력: static + 0M + 0M_goutallier
    출력: label_column (바이너리) + output_columns (회귀)
    """
    def __init__(self, sequential_model, reg_loss_weight=0.3, lr=1e-5, weight_decay=1e-5):
        super().__init__()
        self.sequential_model = sequential_model
        
        # 학습 모드로 변경 (파인튜닝을 위해)
        self.sequential_model.train()
        # 각 서브모델도 학습 모드로 변경
        self.sequential_model.model1.train()
        self.sequential_model.model2.train()
        self.sequential_model.model3.train()
        self.sequential_model.model4.train()
        # 모든 파라미터의 gradient 활성화
        for param in self.sequential_model.parameters():
            param.requires_grad = True
        
        # 회귀 손실 가중치
        self.reg_loss_weight = reg_loss_weight
        self.lr = lr
        self.weight_decay = weight_decay
        
        # 분류용 pos_weight
        self.register_buffer('pos_weight', torch.tensor([cls_pos_weight], dtype=torch.float32))
        
        # 메트릭
        self.train_roc = BinaryAUROC()
        self.val_roc = BinaryAUROC()
        self.test_roc = BinaryAUROC()
        self.val_ap = BinaryAveragePrecision()
        self.test_ap = BinaryAveragePrecision()
        self.train_acc = BinaryAccuracy()
        self.val_acc = BinaryAccuracy()
        self.test_acc = BinaryAccuracy()
        self.train_mse = MeanSquaredError()
        self.val_mse = MeanSquaredError()
        self.test_mse = MeanSquaredError()
        
    def forward(self, x_static, x_0M, x_0M_goutallier):
        """Sequential 모델을 통해 예측 수행 (파인튜닝을 위해 직접 모델 호출)"""
        # Sequential 모델의 각 모델을 직접 호출 (학습 모드 유지)
        # Model 1: static + 0M + 0M_goutallier → 2M
        pred_2M = self.sequential_model.model1(x_static, x_0M, x_0M_goutallier)
        
        # Model 2: static + 0M + 2M + 0M_goutallier → 3M
        pred_3M = self.sequential_model.model2(x_static, x_0M, pred_2M, x_0M_goutallier)
        
        # Model 3: static + 0M + 2M + 3M + 0M_goutallier → 4M
        pred_4M = self.sequential_model.model3(x_static, x_0M, pred_2M, pred_3M, x_0M_goutallier)
        
        # Model 4: static + 0M + 2M + 3M + 4M + 0M_goutallier → 6M + y + 6M_goutallier
        logits, regs, _ = self.sequential_model.model4(x_static, x_0M, pred_2M, pred_3M, pred_4M, x_0M_goutallier)
        
        # 출력 분리: [6M (12) + y (1) + 6M_goutallier (8)]
        pred_6M = regs[:, :seq_features_6M]
        label_logits = logits
        
        # output_columns 예측: 6M의 일부가 output_columns에 해당
        # output_columns는 seq_columns_6M에 포함되어 있으므로 인덱스를 찾아서 추출
        # seq_columns_6M에서 output_columns의 인덱스 찾기
        output_indices = [seq_columns_6M.index(col) for col in output_columns if col in seq_columns_6M]
        if len(output_indices) == len(output_columns):
            output_pred = pred_6M[:, output_indices]
        else:
            # 만약 일부만 매칭되면 앞부분 사용
            output_pred = pred_6M[:, :len(output_columns)]
        
        return label_logits, output_pred
    
    def training_step(self, batch, batch_idx):
        x_static, x_0M, x_0M_goutallier, y_label, y_output = batch
        
        label_logits, output_pred = self.forward(x_static, x_0M, x_0M_goutallier)
        
        # 분류 손실
        clf_loss = F.binary_cross_entropy_with_logits(label_logits, y_label, pos_weight=self.pos_weight)
        
        # 회귀 손실
        reg_loss = F.mse_loss(output_pred, y_output)
        
        # 전체 손실
        loss = clf_loss + self.reg_loss_weight * reg_loss
        
        self.log("train/loss", loss, on_epoch=True, prog_bar=True)
        self.log("train/clf_loss", clf_loss)
        self.log("train/reg_loss", reg_loss)
        
        # 메트릭 업데이트
        probs = label_logits.sigmoid().flatten()
        targets = torch.clamp(y_label.flatten().to(torch.int), 0, 1)
        self.train_roc.update(probs, targets)
        self.train_acc.update(probs, targets)
        self.train_mse.update(output_pred, y_output)
        
        return loss
    
    def validation_step(self, batch, batch_idx):
        x_static, x_0M, x_0M_goutallier, y_label, y_output = batch
        
        label_logits, output_pred = self.forward(x_static, x_0M, x_0M_goutallier)
        
        # 분류 손실
        clf_loss = F.binary_cross_entropy_with_logits(label_logits, y_label, pos_weight=self.pos_weight)
        
        # 회귀 손실
        reg_loss = F.mse_loss(output_pred, y_output)
        
        # 전체 손실
        loss = clf_loss + self.reg_loss_weight * reg_loss
        
        self.log("val/loss", loss, on_epoch=True, prog_bar=True)
        self.log("val/clf_loss", clf_loss)
        self.log("val/reg_loss", reg_loss)
        
        # 메트릭 업데이트
        probs = label_logits.sigmoid().flatten()
        targets = torch.clamp(y_label.flatten().to(torch.int), 0, 1)
        self.val_roc.update(probs, targets)
        self.val_ap.update(probs, targets)
        self.val_acc.update(probs, targets)
        self.val_mse.update(output_pred, y_output)
        
        return loss
    
    def test_step(self, batch, batch_idx):
        x_static, x_0M, x_0M_goutallier, y_label, y_output = batch
        
        label_logits, output_pred = self.forward(x_static, x_0M, x_0M_goutallier)
        
        # 분류 손실
        clf_loss = F.binary_cross_entropy_with_logits(label_logits, y_label, pos_weight=self.pos_weight)
        
        # 회귀 손실
        reg_loss = F.mse_loss(output_pred, y_output)
        
        # 전체 손실
        loss = clf_loss + self.reg_loss_weight * reg_loss
        
        self.log("test/loss", loss, on_epoch=True, prog_bar=True)
        self.log("test/clf_loss", clf_loss)
        self.log("test/reg_loss", reg_loss)
        
        # 메트릭 업데이트
        probs = label_logits.sigmoid().flatten()
        targets = torch.clamp(y_label.flatten().to(torch.int), 0, 1)
        self.test_roc.update(probs, targets)
        self.test_ap.update(probs, targets)
        self.test_acc.update(probs, targets)
        self.test_mse.update(output_pred, y_output)
        
        return loss
    
    def on_train_epoch_end(self):
        self.log("train/roc", self.train_roc.compute())
        self.log("train/acc", self.train_acc.compute())
        self.log("train/mse", self.train_mse.compute())
        self.train_roc.reset()
        self.train_acc.reset()
        self.train_mse.reset()
    
    def on_validation_epoch_end(self):
        self.log("val/roc", self.val_roc.compute())
        self.log("val/ap", self.val_ap.compute())
        self.log("val/acc", self.val_acc.compute())
        self.log("val/mse", self.val_mse.compute())
        self.val_roc.reset()
        self.val_ap.reset()
        self.val_acc.reset()
        self.val_mse.reset()
    
    def on_test_epoch_end(self):
        self.log("test/roc", self.test_roc.compute())
        self.log("test/ap", self.test_ap.compute())
        self.log("test/acc", self.test_acc.compute())
        self.log("test/mse", self.test_mse.compute())
        self.test_roc.reset()
        self.test_ap.reset()
        self.test_acc.reset()
        self.test_mse.reset()
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr, weight_decay=self.weight_decay)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='max', factor=0.5, patience=5
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val/roc"
            }
        }

print("파인튜닝 가능한 Sequential 모델 클래스 정의 완료")


파인튜닝 가능한 Sequential 모델 클래스 정의 완료


In [30]:
# Sequential 모델을 파인튜닝 모델로 래핑
print("\n" + "=" * 80)
print("Sequential 모델 파인튜닝 시작")
print("=" * 80)

# 기존 Sequential 모델을 파인튜닝 가능한 모델로 변환
finetuning_model = SequentialFineTuningModel(
    sequential_model=sequential_model,
    reg_loss_weight=0.3,
    lr=1e-5,
    weight_decay=1e-5
)

# 데이터로더 생성
batch_size_finetuning = 64
trainloader_finetuning = DataLoader(trainset_finetuning, batch_size=batch_size_finetuning, shuffle=True, pin_memory=True)
valloader_finetuning = DataLoader(valset_finetuning, batch_size=batch_size_finetuning)
testloader_finetuning = DataLoader(testset_finetuning, batch_size=batch_size_finetuning)

# 파인튜닝 실행
print("\n파인튜닝 학습 시작...")
trainer_finetuning = L.Trainer(
    max_epochs=100,
    gradient_clip_val=1.0,
    callbacks=[
        ModelCheckpoint(monitor='val/roc', mode='max', save_top_k=1, save_last=False, filename='sequential-finetuning-best', dirpath=CHECKPOINT_DIR),
        CleanupCheckpointCallback(checkpoint_dir=CHECKPOINT_DIR, filename_prefix='sequential-finetuning-best'),
        EarlyStopping(monitor='val/roc', mode='max', patience=10)
    ]
)

trainer_finetuning.fit(finetuning_model, trainloader_finetuning, valloader_finetuning)
print("파인튜닝 학습 완료!")

# 테스트 데이터로 평가
print("\n" + "=" * 80)
print("파인튜닝된 모델 테스트 평가")
print("=" * 80)

test_results_finetuning = trainer_finetuning.test(finetuning_model, testloader_finetuning)

print("\n[테스트 성능]")
print(f"  ROC AUC: {test_results_finetuning[0]['test/roc']:.6f}")
print(f"  AP (Average Precision): {test_results_finetuning[0]['test/ap']:.6f}")
print(f"  Accuracy: {test_results_finetuning[0]['test/acc']:.6f}")
print(f"  MSE (output_columns): {test_results_finetuning[0]['test/mse']:.6f}")

# 상세 정확도 평가
@torch.no_grad()
def evaluate_finetuning_accuracy(model, testloader):
    """파인튜닝된 모델의 정확도 상세 평가"""
    model.eval()
    
    all_label_probs = []
    all_label_targets = []
    all_output_preds = []
    all_output_targets = []
    
    for batch in testloader:
        x_static, x_0M, x_0M_goutallier, y_label, y_output = batch
        label_logits, output_pred = model(x_static, x_0M, x_0M_goutallier)
        
        all_label_probs.append(label_logits.sigmoid())
        all_label_targets.append(y_label)
        all_output_preds.append(output_pred)
        all_output_targets.append(y_output)
    
    # 분류 정확도
    label_probs = torch.cat(all_label_probs, dim=0).flatten()
    label_targets = torch.cat(all_label_targets, dim=0).flatten()
    label_preds = (label_probs > 0.5).int()
    label_targets_int = label_targets.to(torch.int)
    
    accuracy = (label_preds == label_targets_int).float().mean().item()
    
    # 회귀 성능
    output_preds = torch.cat(all_output_preds, dim=0)
    output_targets = torch.cat(all_output_targets, dim=0)
    mse = F.mse_loss(output_preds, output_targets).item()
    mae = F.l1_loss(output_preds, output_targets).item()
    
    return {
        'label_accuracy': accuracy,
        'label_probs': label_probs.cpu().numpy(),
        'label_targets': label_targets_int.cpu().numpy(),
        'label_preds': label_preds.cpu().numpy(),
        'output_mse': mse,
        'output_mae': mae
    }

# 상세 평가 수행
print("\n상세 정확도 평가 중...")
detailed_results = evaluate_finetuning_accuracy(finetuning_model, testloader_finetuning)

print("\n" + "=" * 80)
print("파인튜닝된 모델 최종 성능")
print("=" * 80)
print(f"\n[label_column 정확도]")
print(f"  Accuracy: {detailed_results['label_accuracy']:.6f}")
print(f"  ROC AUC: {test_results_finetuning[0]['test/roc']:.6f}")
print(f"  AP: {test_results_finetuning[0]['test/ap']:.6f}")

print(f"\n[output_columns 회귀 성능]")
print(f"  MSE: {detailed_results['output_mse']:.6f}")
print(f"  MAE: {detailed_results['output_mae']:.6f}")

# 분류 리포트
from sklearn.metrics import classification_report
print(f"\n[분류 리포트]")
print(classification_report(detailed_results['label_targets'], detailed_results['label_preds'], 
                          target_names=['No Retear', 'Retear']))

print("\n" + "=" * 80)
print("파인튜닝 완료!")
print("=" * 80)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
c:\Users\mulso\anaconda3\envs\arcr\lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:751: Checkpoint directory C:\Users\mulso\Documents\GitHub\ARCR-Feedback\checkpoint exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



Sequential 모델 파인튜닝 시작

파인튜닝 학습 시작...



   | Name             | Type                   | Params | Mode 
---------------------------------------------------------------------
0  | sequential_model | SequentialModel        | 1.3 M  | train
1  | train_roc        | BinaryAUROC            | 0      | train
2  | val_roc          | BinaryAUROC            | 0      | train
3  | test_roc         | BinaryAUROC            | 0      | train
4  | val_ap           | BinaryAveragePrecision | 0      | train
5  | test_ap          | BinaryAveragePrecision | 0      | train
6  | train_acc        | BinaryAccuracy         | 0      | train
7  | val_acc          | BinaryAccuracy         | 0      | train
8  | test_acc         | BinaryAccuracy         | 0      | train
9  | train_mse        | MeanSquaredError       | 0      | train
10 | val_mse          | MeanSquaredError       | 0      | train
11 | test_mse         | MeanSquaredError       | 0      | train
---------------------------------------------------------------------
1.3 M     Trainable params


Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 39.84it/s]

c:\Users\mulso\anaconda3\envs\arcr\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:433: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


c:\Users\mulso\anaconda3\envs\arcr\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


Epoch 11: 100%|██████████| 113/113 [00:03<00:00, 30.24it/s, v_num=10, train/loss_step=0.861, val/loss=0.825, train/loss_epoch=0.854]
파인튜닝 학습 완료!

파인튜닝된 모델 테스트 평가


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\mulso\anaconda3\envs\arcr\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:433: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 44.54it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test/acc            0.5699999928474426
         test/ap            0.6076286435127258
      test/clf_loss          0.812909722328186
        test/loss           1.4553276300430298
        test/mse            2.1413934230804443
      test/reg_loss         2.1413931846618652
        test/roc            0.5879999995231628
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

[테스트 성능]
  ROC AUC: 0.588000
  AP (Average Precision): 0.607629
  Accuracy: 0.570000
  MSE (output_columns): 2.141393

상세 정확도 평가 중...

파인튜닝된 모델 최종 성능

[label_column 정확도]
  Accuracy: 0.570000
  RO